In [3]:
import dostools
import importlib
import numpy as np
import pickle
import torch
import sys
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import matplotlib
import time
import scipy 
import copy
import ase
import ase.io
torch.set_default_dtype(torch.float64) 
# %matplotlib notebook
matplotlib.rcParams['figure.figsize'] = (10, 10)

## Loading Data

In [216]:
import dostools.datasets.data as data
import dostools.utils.utils as utils

with torch.no_grad():
#     sigma = 0.3
#     structures = data.load_structures(":")
#     n_structures = len(structures) #total number of structures
#     for structure in structures:#implement periodicity
#         structure.wrap(eps = 1e-12) 
#     n_atoms = np.zeros(n_structures, dtype = int) #stores number of atoms in each structures
#     for i in range(n_structures):
#         n_atoms[i] = len(structures[i])
        
    xdos = torch.tensor(data.load_xdos())
    
    total_dos3 = torch.load("./total_ldos3.pt")
    total_dos1 = torch.load("./total_ldos1.pt")
    
    surface_dos3 = torch.load("./surface_ldos3.pt")
    surface_dos1 = torch.load("./surface_ldos1.pt")
    
    surface_aligned_dos3 = torch.load("./surface_aligned_dos3.pt")
    surface_aligned_dos1 = torch.load("./surface_aligned_dos1.pt")
    
    bulk_dos3 = torch.load("./bulk_ldos3.pt")
    bulk_dos1 = torch.load("./bulk_ldos1.pt")
    
    total_aligned_dos3 = torch.load("./total_aligned_dos3.pt")
    total_aligned_dos1 = torch.load("./total_aligned_dos1.pt")
    
    surface_soap = torch.load("./surface_soap.pt")
    bulk_soap = torch.load("./bulk_soap.pt")
    total_soap = torch.load("./total_soap.pt")
    
    surface_kernel_30 = torch.load("./surface_kernel_30.pt")
    surface_kMM_30 = torch.load("./surface_kMM_30.pt")
    
    bulk_kernel_200 = torch.load("./bulk_kernel_200.pt")
    bulk_kMM_200 = torch.load("./bulk_kMM_200.pt")
    
    bulk_kernel_100 = torch.load("./bulk_kernel_100.pt")
    bulk_kMM_100 = torch.load("./bulk_kMM_100.pt")
    
    total_kernel_100 = torch.load("./total_kernel_100.pt")
    total_kMM_100 = torch.load("./total_kMM_100.pt")
    
    total_kernel_150 = torch.load("./total_kernel_150.pt")
    total_kMM_150 = torch.load("./total_kMM_150.pt")
    
    

## Generate all train-test indexes

In [6]:
def generate_train_test_split(n_samples):
    n_structures = n_samples
    np.random.seed(0)
    n_train = int(0.8 * n_structures)
    train_index = np.arange(n_structures)
    np.random.shuffle(train_index)
    test_index = train_index[n_train:]
    train_index = train_index[:n_train]
    
    return train_index, test_index

def generate_biased_train_test_split(n_samples):
    #Assumes 100 amorphous structures at the end
    n_structures = n_samples
    amorph_train = np.arange(n_samples-100, n_samples,1)
    np.random.seed(0)
    np.random.shuffle(amorph_train)
    
    amorph_test = amorph_train[:80]
    amorph_train = amorph_train[80:]

    n_structures = n_samples - 100
    np.random.seed(0)
    n_train = int(0.8 * n_samples)-20
    remaining_train_index = np.arange(n_structures)
    np.random.shuffle(remaining_train_index)

    remaining_test_index = remaining_train_index[n_train:]
    remaining_train_index = remaining_train_index[:n_train]

    biased_train_index = np.concatenate([remaining_train_index, amorph_train])
    biased_test_index = np.concatenate([remaining_test_index, amorph_test])
    
    return biased_train_index, biased_test_index

def generate_surface_holdout_split(n_samples):
    #Assumes that we are using the 110 surfaces for test which are located at 673 + 31st-57th index
    #26 structures
    
    n_test = int(0.2 * n_samples) - 26
    n_train = n_samples - n_test
    
    remaining_indexes = np.concatenate([np.arange(673+31), np.arange(673+57,n_samples,1)])
    indexes_110 = np.arange(673+31, 673+57,1)
    np.random.seed(0)
    
    np.random.shuffle(remaining_indexes)
    
    remaining_test_index = remaining_indexes[n_train:]
    remaining_train_index = remaining_indexes[:n_train]
    
    total_train_index = remaining_train_index
    total_test_index = np.concatenate([remaining_test_index, indexes_110])
    
    return total_train_index, total_test_index
    
def surface_holdout(n_samples):
    test_index = np.arange(31,57,1)
    train_index = np.concatenate([np.arange(31), np.arange(57, n_samples)])
    
    return train_index, test_index

n_surfaces = 154
n_bulkstructures = 773
n_total_structures = 773 + 154


surface_train_index, surface_test_index = generate_train_test_split(n_surfaces)
bulk_train_index, bulk_test_index = generate_train_test_split(n_bulkstructures)
total_train_index, total_test_index = generate_train_test_split(n_total_structures)
surface_holdout_train_index, surface_holdout_test_index = surface_holdout(n_surfaces)
bulk_biased_train_index, bulk_biased_test_index = generate_biased_train_test_split(n_bulkstructures)
total_biased_train_index, total_biased_test_index = generate_biased_train_test_split(n_total_structures)
holdout_train_index, holdout_test_index = generate_surface_holdout_split(n_total_structures)

## Determine Cutoff for truncated data

In [447]:
cutoff - 3

tensor(-4.7117)

In [310]:
# Fermi level
import ase

T_0 = 200
beta_0 = 1 / (ase.units.kB * T_0) # inverse temperature
efermi = torch.zeros(n_total_structures)
for i in range(n_total_structures):
    efermi[i] = torch.tensor(utils.getmu(total_dos1[i], beta_0, xdos, n=4))
cutoff = torch.max(efermi) + 3
cutoff_index = torch.searchsorted(xdos, cutoff)


In [443]:
cutoff_index

tensor(487)

## Opt Discrete shift

In [7]:
#Generate shifted data
def shifted_ldos(ldos, xdos, shift): 
    xdos_step = xdos[1] - xdos[0]
    shifted_ldos = torch.zeros_like(ldos)
    if len(ldos.shape) > 1:
        xdos_shift = torch.round(shift/xdos_step).int()
        for i in range(len(ldos)):
            if xdos_shift[i] > 0:
                shifted_ldos[i] = torch.nn.functional.pad(ldos[i,:-1*xdos_shift[i]], (xdos_shift[i],0))
            elif xdos_shift[i] < 0:
                shifted_ldos[i] = torch.nn.functional.pad(ldos[i,(-1*xdos_shift[i]):], (0,(-1*xdos_shift[i])))
            else:
                shifted_ldos[i] = ldos[i]
    else:        
        xdos_shift = int(torch.round(shift/xdos_step))
        if xdos_shift > 0:
            shifted_ldos = torch.nn.functional.pad(ldos[:-1*xdos_shift], (xdos_shift,0))
        elif xdos_shift < 0:
            shifted_ldos = torch.nn.functional.pad(ldos[(-1*xdos_shift):], (0,(-1*xdos_shift)))
        else:
            shifted_ldos = ldos
    return shifted_ldos



In [239]:
from scipy.signal import convolve, correlate, correlation_lags
def find_optimal_discrete_shift(y1, y2):
    if y2.shape == y1.shape and len(y1.shape) == 2:
        shift = []
        for i in range(y2.shape[0]):
            corr = correlate(y1[i], y2[i], mode='full')
            shift_i = np.argmax(corr) - len(y2[i]) + 1   
            shift.append(shift_i)
        
        
    elif y2.shape == y1.shape and len(y1.shape) == 1:
        corr = correlate(y1, y2, mode='full')
        shift = np.argmax(corr) - len(y2) + 1   
    else:
        print ("input shapes are not the same")
        raise Exception
    return shift


In [ ]:
test_ldos0 = shifted_ldos(ldos3, xdos, torch.ones(1039))
optshift = find_optimal_shift(np.array(ldos3), np.array(test_ldos0))

## Training Functions

### Analytical

In [18]:
def normal_reg_train_A(feat, target, train_index, test_index, reg):
    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])
    Features = features[train_index]#Silicon.Features['structure_avekerneldescriptors'][train_index]
    test_features = features[test_index]#Silicon.Features['structure_avekerneldescriptors'][test_index]
    Target = target[train_index]
    test_target = target[test_index]
    n_col = Features.shape[1]
    regularization = reg
    reg = regularization * torch.eye(n_col)
    reg[-1, -1] = 0
    A = torch.vstack([Features, reg])
    b = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])
    weights = torch.linalg.lstsq(A, b, rcond=1e-10).solution
    pred = Features @ weights 
    test_pred = test_features @ weights

    loss_dos = loss.t_get_rmse(pred, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(test_pred, test_target, xdos, perc = True)
    
    return weights, loss_dos, test_loss_dos

def kernel_reg_train_A(feat, target, train_index, test_index, reg, kMM):
    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])
    Features = features[train_index]#Silicon.Features['structure_avekerneldescriptors'][train_index]
    test_features = features[test_index]#Silicon.Features['structure_avekerneldescriptors'][test_index]
    Target = target[train_index]
    test_target = target[test_index]
    n_col = Features.shape[1]
    regularization = reg
    rtkMM = scipy.linalg.sqrtm(kMM)
    reg = torch.hstack([(torch.tensor(regularization * rtkMM)), torch.zeros(kMM.shape[0]).view(-1,1)])
    reg = torch.vstack([reg, torch.zeros(n_col)])
    # reg[-1, -1] = 0
    A = torch.vstack([Features, reg])
    b = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])

    weights = torch.linalg.lstsq(A, b, driver = "gelsd", rcond = 1e-10).solution
    pred = Features @ weights 
    test_pred = test_features @ weights

    loss_dos = loss.t_get_rmse(pred, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(test_pred, test_target, xdos, perc = True)
    
    return weights, loss_dos, test_loss_dos


### Normal Reg

In [237]:
from dostools.loss import loss

def normal_reg_train_L(feat, target, train_index, test_index, regularization, n_epochs, lr):
    
    patience = 20
    index = train_index
    t_index = test_index
    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])
    Features = features[index]
    t_Features = features[t_index]
    n_col = Features.shape[1]
    Target = target[index]
    t_Target = target[t_index]
    reg = regularization * torch.eye(n_col)
    reg[-1, -1] = 0
    reg_features = torch.vstack([Features, reg])
    reg_target = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])
    


    weights = torch.nn.Parameter(torch.rand(Features.shape[1], Target.shape[1])- 0.5)
    opt = torch.optim.LBFGS([weights], lr = lr, line_search_fn = "strong_wolfe", tolerance_grad = 1e-20, tolerance_change = 1-20, history_size = 200)
    pbar = tqdm(range(n_epochs))
    current_rmse = torch.tensor(100)
    pred_loss = torch.tensor(100)
    prev_loss = torch.tensor(100)
    best_mse = torch.tensor(100)
    trigger = 0
    for epoch in pbar:
        pbar.set_description(f"Epoch: {epoch}")
        pbar.set_postfix(pred_loss = pred_loss.item(), lowest_mse = best_mse.item(), trigger = trigger)
        def closure():
            opt.zero_grad()
            pred_i = reg_features @ weights
            loss_i = loss.t_get_mse(pred_i, reg_target)
            loss_i.backward()
            return loss_i
        opt.step(closure)

        with torch.no_grad():
            preds = Features @ weights
            epoch_rmse = loss.t_get_rmse(preds, Target, xdos, perc = True)
            epoch_mse = loss.t_get_mse(preds, Target, xdos)


            pred_loss = epoch_rmse

            if epoch_mse < best_mse:
                best_mse = epoch_mse
                best_state = weights.clone()

            if epoch_mse < prev_loss * ( 1 + 1e-3):
                trigger =0
            else:
                trigger +=1 
                if trigger >= patience:
                    weights = best_state
                    opt = torch.optim.Adam([weights], lr = opt.param_groups[0]['lr'], weight_decay = 0)

            epoch_mse = prev_loss


    

    final_preds = Features @ best_state 
    final_t_preds = t_Features @ best_state

    loss_dos = loss.t_get_rmse(final_preds, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(final_t_preds, t_Target, xdos, perc = True)
    return best_state, loss_dos, test_loss_dos

def normal_reg_train_Ad(feat, target, train_index, test_index, regularization, n_epochs, batch_size, lr):
    index = train_index
    t_index = test_index

    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])

    Sampler = torch.utils.data.RandomSampler(index, replacement = False)
    Batcher = torch.utils.data.BatchSampler(Sampler, batch_size, False)

    Features = features[index]
    t_Features = features[t_index]
    n_col = Features.shape[1]


    Target = target[index]
    t_Target = target[t_index]


    # reg_features = torch.vstack([Features, reg])
    # reg_target = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])


    reg = regularization * torch.eye(n_col)
    reg[-1, -1] = 0


    weights = torch.nn.Parameter((torch.rand(Features.shape[1], Target.shape[1])- 0.5))
    opt = torch.optim.Adam([weights], lr = lr, weight_decay = 0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor = 0.1, patience = 500, threshold = 1e-7, min_lr = 1e-8)

    pbar = tqdm(range(n_epochs))

    current_rmse = torch.tensor(100)
    pred_loss = torch.tensor(100)
    prev_loss = torch.tensor(100)
    best_mse = torch.tensor(100)
    trigger = 0
    for epoch in pbar:
        pbar.set_description(f"Epoch: {epoch}")
        pbar.set_postfix(pred_loss = pred_loss.item(), lowest_mse = best_mse.item(), trigger = trigger)
        for i_batch in Batcher:
            def closure():
                opt.zero_grad()
                reg_features_i = torch.vstack([Features[i_batch], reg])
                target_i = torch.vstack([Target[i_batch], torch.zeros(n_col, Target.shape[1])])
                pred_i = reg_features_i @ weights
                loss_i = loss.t_get_mse(pred_i, target_i)
                loss_i.backward()
                return loss_i
            opt.step(closure)

        with torch.no_grad():
            preds = Features @ weights
            epoch_rmse = loss.t_get_rmse(preds, Target, xdos, perc = True)
            epoch_mse = loss.t_get_mse(preds, Target, xdos)


            pred_loss = epoch_rmse

            if epoch_mse < best_mse:
                best_mse = epoch_mse
                best_state = weights.clone()

            if epoch_mse < prev_loss * ( 1 + 1e-3):
                trigger =0
            else:
                trigger +=1 
                if trigger >= patience:
                    weights = best_state
                    opt = torch.optim.Adam([weights], lr = opt.param_groups[0]['lr'], weight_decay = 0)

            epoch_mse = prev_loss

            scheduler.step(epoch_mse)

            if Batcher.batch_size > 1024:
                break

            if opt.param_groups[0]['lr'] < 1e-4:
                Batcher.batch_size *= 2
                opt.param_groups[0]['lr'] = lr
                print ("The batch_size is now: ", Batcher.batch_size)

    

    final_preds = Features @ best_state 
    final_t_preds = t_Features @ best_state

    loss_dos = loss.t_get_rmse(final_preds, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(final_t_preds, t_Target, xdos, perc = True)
    return best_state, loss_dos, test_loss_dos
        

In [456]:
a = 3 \
+ 7
print (a)

10


### Kernel reg

In [450]:
def kernel_reg_train_Ad(feat, target, train_index, test_index, kMM, regularization, n_epochs, batch_size, lr):
    index = train_index
    t_index = test_index
    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])
    Features = features[index]
    t_Features = features[t_index]
    n_col = Features.shape[1]
    Target = target[index]
    t_Target = target[t_index]
    Sampler = torch.utils.data.RandomSampler(index, replacement = False)
    Batcher = torch.utils.data.BatchSampler(Sampler, batch_size, False)
    rtkMM = scipy.linalg.sqrtm(kMM)
    reg = torch.hstack([(torch.tensor(regularization * rtkMM)), torch.zeros(kMM.shape[0]).view(-1,1)])
    reg = torch.vstack([reg, torch.zeros(n_col)])

    # reg_features = torch.vstack([Features, reg])
    # reg_target = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])
    weights = torch.nn.Parameter(torch.rand(Features.shape[1], Target.shape[1])- 0.5) 

    # weights = torch.nn.Parameter((torch.rand(Features.shape[1], Target.shape[1])- 0.5) * 1e-7)
    opt = torch.optim.Adam([weights], lr = 1e-3, weight_decay = 0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor = 0.1, patience = 200, threshold = 1e-5, min_lr = 1e-8)
    pbar = tqdm(range(n_epochs))
    
    current_rmse = torch.tensor(100)
    pred_loss = torch.tensor(100)
    prev_loss = torch.tensor(100)
    best_mse = torch.tensor(100)
    trigger = 0
    for epoch in pbar:
        pbar.set_description(f"Epoch: {epoch}")
        pbar.set_postfix(pred_loss = pred_loss.item(), lowest_mse = best_mse.item(), trigger = trigger)
        for i_batch in Batcher:
            def closure():
                opt.zero_grad()
                reg_features_i = torch.vstack([Features[i_batch], reg])
                target_i = torch.vstack([Target[i_batch], torch.zeros(n_col, Target.shape[1])])
                pred_i = reg_features_i @ weights
                loss_i = loss.t_get_mse(pred_i, target_i)
                loss_i.backward()
                return loss_i
            opt.step(closure)

        with torch.no_grad():
            preds = Features @ weights
            epoch_rmse = loss.t_get_rmse(preds, Target, xdos, perc = True)
            epoch_mse = loss.t_get_mse(preds, Target, xdos)


            pred_loss = epoch_rmse

            if epoch_mse < best_mse:
                best_mse = epoch_mse
                best_state = weights.clone()

            if epoch_mse < prev_loss * ( 1 + 1e-3):
                trigger =0
            else:
                trigger +=1 
                if trigger >= patience:
                    weights = best_state
                    opt = torch.optim.Adam([weights], lr = opt.param_groups[0]['lr'], weight_decay = 0)

            epoch_mse = prev_loss

            scheduler.step(epoch_mse)

            if Batcher.batch_size > 1024:
                break

            if opt.param_groups[0]['lr'] < 1e-4:
                Batcher.batch_size *= 2
                opt.param_groups[0]['lr'] = lr
                print ("The batch_size is now: ", Batcher.batch_size)


    

    final_preds = Features @ best_state 
    final_t_preds = t_Features @ best_state

    loss_dos = loss.t_get_rmse(final_preds, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(final_t_preds, t_Target, xdos, perc = True)
    
    return best_state, loss_dos, test_loss_dos


def kernel_reg_train_L(feat, target, train_index, test_index, kMM, regularization, n_epochs, lr):
    index = train_index
    t_index = test_index
    features = torch.hstack([feat, torch.ones(feat.shape[0]).view(-1,1)])
    Features = features[index]
    t_Features = features[t_index]
    n_col = Features.shape[1]
    Target = target[index]
    t_Target = target[t_index]
    rtkMM = scipy.linalg.sqrtm(kMM)
    reg = torch.hstack([(torch.tensor(regularization * rtkMM)), torch.zeros(kMM.shape[0]).view(-1,1)])
    reg = torch.vstack([reg, torch.zeros(n_col)])

    reg_features = torch.vstack([Features, reg])
    reg_target = torch.vstack([Target, torch.zeros(n_col,Target.shape[1])])
    weights = torch.nn.Parameter(torch.rand(Features.shape[1], Target.shape[1])- 0.5) 
    opt = torch.optim.LBFGS([weights], lr = lr, line_search_fn = "strong_wolfe", tolerance_grad = 1e-20, tolerance_change = 1-20, history_size = 200)

    current_rmse = torch.tensor(100)
    pred_loss = torch.tensor(100)
    prev_loss = torch.tensor(100)
    best_mse = torch.tensor(100)
    trigger = 0
    pbar = tqdm(range(n_epochs))
    for epoch in pbar:
        pbar.set_description(f"Epoch: {epoch}")
        pbar.set_postfix(pred_loss = pred_loss.item(), lowest_mse = best_mse.item(), trigger = trigger)
        def closure():
            opt.zero_grad()
            pred_i = reg_features @ weights
            loss_i = loss.t_get_mse(pred_i, reg_target)
            loss_i.backward()
            return loss_i
        opt.step(closure)

        with torch.no_grad():
            preds = Features @ weights
            epoch_rmse = loss.t_get_rmse(preds, Target, xdos, perc = True)
            epoch_mse = loss.t_get_mse(preds, Target, xdos)


            pred_loss = epoch_rmse

            if epoch_mse < best_mse:
                best_mse = epoch_mse
                best_state = weights.clone()

            if epoch_mse < prev_loss * ( 1 + 1e-3):
                trigger =0
            else:
                trigger +=1 
                if trigger >= patience:
                    weights = best_state
                    opt = torch.optim.Adam([weights], lr = opt.param_groups[0]['lr'], weight_decay = 0)

            epoch_mse = prev_loss

    final_preds = Features @ best_state 
    final_t_preds = t_Features @ best_state

    loss_dos = loss.t_get_rmse(final_preds, Target, xdos, perc = True)
    test_loss_dos = loss.t_get_rmse(final_t_preds, t_Target, xdos, perc = True)
    
    return best_state, loss_dos, test_loss_dos
        

## Baseline Linalg Models - Full data

### Surfaces

#### Surfaces - 0.3 SOAP

In [25]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos3, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos3, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Holdout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 28.31 for SOAP
The test error is 43.85 for SOAP
Holdout:
The train error is 49.36 for SOAP
The test error is 126.7 for SOAP


#### Surfaces - 0.1 SOAP

In [24]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos1, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos1, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Goldout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 39.1 for SOAP
The test error is 58.12 for SOAP
Biased
The train error is 59.98 for SOAP
The test error is 132.5 for SOAP


#### Surfaces - 0.3 Kernel

In [28]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos3, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            regularization, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 24.47 for n_refs = 30
The test error is 38.6 for n_refs = 30
Biased
The train error is 27.03 for n_refs = 30
The test error is 109.9 for n_refs = 30


#### Surfaces - 0.1 Kernel

In [30]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos1, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 36.07 for n_refs = 30
The test error is 55.46 for n_refs = 30
Biased
The train error is 53.3 for n_refs = 30
The test error is 122.8 for n_refs = 30


### Surfaces Aligned

####  Surfaces Aligned - 0.3 SOAP

In [215]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Holdout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 29.29 for SOAP
The test error is 44.48 for SOAP
Holdout:
The train error is 49.82 for SOAP
The test error is 122.8 for SOAP


####  Surfaces Aligned - 0.1 SOAP

In [217]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos1, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Goldout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 40.1 for SOAP
The test error is 59.12 for SOAP
Goldout:
The train error is 60.68 for SOAP
The test error is 130.3 for SOAP


####  Surfaces Aligned - 0.3 Kernel

In [196]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos3, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            regularization, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 25.79 for n_refs = 30
The test error is 39.91 for n_refs = 30
Biased
The train error is 28.51 for n_refs = 30
The test error is 110.8 for n_refs = 30


####  Surfaces Aligned - 0.1 Kernel

In [235]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos1, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 37.3 for n_refs = 30
The test error is 56.96 for n_refs = 30
Biased
The train error is 54.48 for n_refs = 30
The test error is 121.1 for n_refs = 30


### Bulk

#### Bulk - 0.3 SOAP 

In [32]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos3,
                                                          bulk_train_index, bulk_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos3,
                                                          bulk_biased_train_index, bulk_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 14.79 for SOAP
The test error is 20.91 for SOAP
Biased
The train error is 14.52 for SOAP
The test error is 23.67 for SOAP


#### Bulk- 0.1 SOAP 

In [33]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos1,
                                                          bulk_train_index, bulk_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos1,
                                                          bulk_biased_train_index, bulk_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 32.97 for SOAP
The test error is 40.62 for SOAP
Biased
The train error is 32.89 for SOAP
The test error is 44.41 for SOAP


#### Bulk - 0.3 Kernel

In [39]:
regularization = 1e-1
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_200, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            regularization, bulk_kMM_200)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 200".format(loss_dos))
print ("The test error is {:.4} for n_refs = 200".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_100, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            regularization, bulk_kMM_100)
print ("Biased")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 12.41 for n_refs = 200
The test error is 18.0 for n_refs = 200
Biased
The train error is 16.86 for n_refs = 100
The test error is 22.53 for n_refs = 100


#### Bulk - 0.1 Kernel 

In [40]:
regularization = 1e-1
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_200, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            regularization, bulk_kMM_200)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 200".format(loss_dos))
print ("The test error is {:.4} for n_refs = 200".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_100, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            regularization, bulk_kMM_100)
print ("Biased")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 30.49 for n_refs = 200
The test error is 37.77 for n_refs = 200
Biased
The train error is 35.62 for n_refs = 100
The test error is 42.45 for n_refs = 100


### Total

#### Total - 0.3 SOAP

In [44]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.3")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.3
Regularization: 0.01
Unbiased
The train error is 19.48 for SOAP
The test error is 23.31 for SOAP
Biased
The train error is 18.78 for SOAP
The test error is 27.53 for SOAP
Holdout
The train error is 17.71 for SOAP
The test error is 33.55 for SOAP


#### Total - 0.1 SOAP

In [43]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 32.58 for SOAP
The test error is 37.84 for SOAP
Biased
The train error is 32.02 for SOAP
The test error is 42.22 for SOAP
Holdout
The train error is 31.9 for SOAP
The test error is 44.47 for SOAP


#### Total - 0.3 Kernel

In [47]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos3,
                                                            total_train_index, total_test_index,
                                                            regularization, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            regularization, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_100, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            regularization, total_kMM_100)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 12.44 for n_refs = 150
The test error is 18.07 for n_refs = 150
Biased
The train error is 11.89 for n_refs = 150
The test error is 21.96 for n_refs = 150
Holdout
The train error is 13.46 for n_refs = 100
The test error is 27.5 for n_refs = 100


#### Total - 0.1 Kernel

In [82]:
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-1, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-1, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, total_kMM_150)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 31.09 for n_refs = 150
The test error is 34.86 for n_refs = 150
Biased
The train error is 30.76 for n_refs = 150
The test error is 38.33 for n_refs = 150
Holdout
The train error is 26.7 for n_refs = 150
The test error is 38.73 for n_refs = 150


### Total Aligned

#### Total Aligned - 0.3 SOAP

In [133]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.3")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.3
Regularization: 0.01
Unbiased
The train error is 18.12 for SOAP
The test error is 22.14 for SOAP
Biased
The train error is 17.46 for SOAP
The test error is 26.29 for SOAP
Holdout
The train error is 16.88 for SOAP
The test error is 29.8 for SOAP


#### Total Aligned - 0.1 SOAP

In [134]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 31.92 for SOAP
The test error is 37.29 for SOAP
Biased
The train error is 31.39 for SOAP
The test error is 41.68 for SOAP
Holdout
The train error is 31.56 for SOAP
The test error is 41.96 for SOAP


#### Total Aligned - 0.3 Kernel

In [135]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            regularization, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            regularization, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_100, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            regularization, total_kMM_100)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 11.94 for n_refs = 150
The test error is 17.52 for n_refs = 150
Biased
The train error is 11.42 for n_refs = 150
The test error is 21.28 for n_refs = 150
Holdout
The train error is 13.06 for n_refs = 100
The test error is 24.41 for n_refs = 100


#### Total Aligned - 0.1 Kernel

In [136]:
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-1, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-1, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, total_kMM_150)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 30.68 for n_refs = 150
The test error is 34.5 for n_refs = 150
Biased
The train error is 30.37 for n_refs = 150
The test error is 37.95 for n_refs = 150
Holdout
The train error is 26.58 for n_refs = 150
The test error is 37.09 for n_refs = 150


## Baseline GD model - Adam 

### Surfaces

#### Surfaces - 0.3 SOAP

In [67]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos3, surface_train_index, surface_test_index, 1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1011:  10%|███████████                                                                                                   | 1008/10000 [00:44<04:49, 31.03it/s, lowest_mse=0.0254, pred_loss=60.3, trigger=0]

The batch_size is now:  32


Epoch: 2017:  20%|██████████████████████▏                                                                                       | 2012/10000 [01:09<02:58, 44.86it/s, lowest_mse=0.0146, pred_loss=45.8, trigger=0]

The batch_size is now:  64


Epoch: 3027:  30%|█████████████████████████████████▏                                                                            | 3018/10000 [01:24<01:34, 73.50it/s, lowest_mse=0.0121, pred_loss=41.7, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6025/10000 [01:54<00:41, 96.63it/s, lowest_mse=0.00924, pred_loss=36.4, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [02:04<00:53, 56.24it/s, lowest_mse=0.00879, pred_loss=35.5, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 35.5 for SOAP
The test error is 48.62 for SOAP


In [69]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos3,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1008:  10%|███████████                                                                                                   | 1006/10000 [01:08<08:37, 17.38it/s, lowest_mse=0.0303, pred_loss=73.8, trigger=0]

The batch_size is now:  32


Epoch: 2011:  20%|██████████████████████                                                                                        | 2010/10000 [01:46<04:44, 28.12it/s, lowest_mse=0.0205, pred_loss=60.7, trigger=0]

The batch_size is now:  64


Epoch: 3023:  30%|█████████████████████████████████▏                                                                            | 3018/10000 [02:09<02:00, 58.04it/s, lowest_mse=0.0175, pred_loss=56.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6030/10000 [02:46<00:42, 93.18it/s, lowest_mse=0.0143, pred_loss=50.6, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [02:57<01:15, 39.61it/s, lowest_mse=0.0139, pred_loss=50.1, trigger=0]

The batch_size is now:  2048
Adam Holdout
The train error is 50.09 for SOAP
The test error is 130.8 for SOAP


#### Surfaces - 0.1 SOAP

In [70]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos1, 
                                                       surface_train_index, surface_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1011:  10%|███████████                                                                                                   | 1009/10000 [00:41<05:00, 29.93it/s, lowest_mse=0.0566, pred_loss=67.9, trigger=0]

The batch_size is now:  32


Epoch: 2019:  20%|██████████████████████▏                                                                                       | 2017/10000 [01:04<02:19, 57.26it/s, lowest_mse=0.0405, pred_loss=57.4, trigger=0]

The batch_size is now:  64


Epoch: 3027:  30%|█████████████████████████████████▏                                                                            | 3020/10000 [01:19<01:35, 73.26it/s, lowest_mse=0.0357, pred_loss=53.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6035:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6026/10000 [01:47<00:36, 110.05it/s, lowest_mse=0.029, pred_loss=48.6, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:57<00:50, 59.55it/s, lowest_mse=0.0276, pred_loss=47.4, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 47.43 for SOAP
The test error is 61.55 for SOAP


In [71]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos1,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1012:  10%|███████████                                                                                                   | 1010/10000 [00:38<04:21, 34.38it/s, lowest_mse=0.0653, pred_loss=79.4, trigger=0]

The batch_size is now:  32


Epoch: 2020:  20%|██████████████████████▏                                                                                       | 2016/10000 [01:01<02:17, 58.10it/s, lowest_mse=0.0514, pred_loss=70.5, trigger=0]

The batch_size is now:  64


Epoch: 3030:  30%|█████████████████████████████████▌                                                                             | 3022/10000 [01:14<01:16, 91.21it/s, lowest_mse=0.046, pred_loss=66.7, trigger=0]

...

The batch_size is now:  512


Epoch: 6038:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6027/10000 [01:42<00:33, 118.00it/s, lowest_mse=0.0395, pred_loss=61.8, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:51<00:47, 62.91it/s, lowest_mse=0.0387, pred_loss=61.1, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 61.12 for SOAP
The test error is 138.2 for SOAP


#### Surfaces - 0.3 Kernel

In [72]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 828:   8%|█████████▏                                                                                                       | 817/10000 [00:11<01:45, 87.02it/s, lowest_mse=0.0759, pred_loss=104, trigger=0]

The batch_size is now:  32


Epoch: 1643:  16%|█████████████████▋                                                                                           | 1627/10000 [00:18<01:03, 132.73it/s, lowest_mse=0.0558, pred_loss=89.4, trigger=0]

The batch_size is now:  64


Epoch: 2457:  24%|██████████████████████████▋                                                                                  | 2443/10000 [00:22<00:39, 190.62it/s, lowest_mse=0.0454, pred_loss=80.6, trigger=0]

...

The batch_size is now:  512


Epoch: 4869:  49%|████████████████████████████████████████████████████▉                                                        | 4857/10000 [00:33<00:23, 223.27it/s, lowest_mse=0.0291, pred_loss=64.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:37<00:29, 149.60it/s, lowest_mse=0.0257, pred_loss=60.7, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 60.68 for KERNEL
The test error is 66.62 for KERNEL


In [73]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 825:   8%|█████████▏                                                                                                       | 816/10000 [00:11<01:55, 79.44it/s, lowest_mse=0.0617, pred_loss=105, trigger=0]

The batch_size is now:  32


Epoch: 1641:  16%|█████████████████▋                                                                                           | 1626/10000 [00:19<01:07, 123.74it/s, lowest_mse=0.0471, pred_loss=92.1, trigger=0]

The batch_size is now:  64


Epoch: 2455:  24%|██████████████████████████▌                                                                                  | 2436/10000 [00:24<00:43, 174.91it/s, lowest_mse=0.0391, pred_loss=83.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4862:  49%|████████████████████████████████████████████████████▉                                                        | 4860/10000 [00:36<00:27, 186.11it/s, lowest_mse=0.0274, pred_loss=70.2, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:40<00:31, 138.95it/s, lowest_mse=0.0244, pred_loss=66.3, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 66.24 for KERNEL
The test error is 180.0 for KERNEL


#### Surfaces - 0.1 Kernel

In [74]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos1,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 825:   8%|█████████▎                                                                                                        | 816/10000 [00:12<01:58, 77.42it/s, lowest_mse=0.123, pred_loss=100, trigger=0]

The batch_size is now:  32


Epoch: 1638:  16%|█████████████████▊                                                                                            | 1624/10000 [00:20<01:14, 112.50it/s, lowest_mse=0.102, pred_loss=91.1, trigger=0]

The batch_size is now:  64


Epoch: 2458:  24%|██████████████████████████▋                                                                                  | 2448/10000 [00:25<00:38, 197.07it/s, lowest_mse=0.0906, pred_loss=85.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4868:  49%|█████████████████████████████████████████████████████                                                        | 4863/10000 [00:36<00:23, 219.90it/s, lowest_mse=0.0701, pred_loss=75.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:39<00:30, 141.58it/s, lowest_mse=0.0647, pred_loss=72.6, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 72.56 for KERNEL
The test error is 77.81 for KERNEL


In [75]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-1, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 828:   8%|█████████▎                                                                                                       | 822/10000 [00:10<01:39, 92.21it/s, lowest_mse=0.0607, pred_loss=104, trigger=0]

The batch_size is now:  32


Epoch: 1639:  16%|█████████████████▋                                                                                           | 1628/10000 [00:18<01:10, 117.92it/s, lowest_mse=0.0464, pred_loss=91.4, trigger=0]

The batch_size is now:  64


Epoch: 2453:  25%|██████████████████████████▋                                                                                  | 2452/10000 [00:23<00:43, 175.40it/s, lowest_mse=0.0385, pred_loss=83.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4863:  49%|████████████████████████████████████████████████████▉                                                        | 4858/10000 [00:36<00:27, 190.17it/s, lowest_mse=0.0268, pred_loss=69.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:39<00:30, 142.38it/s, lowest_mse=0.0239, pred_loss=65.6, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 65.54 for KERNEL
The test error is 176.8 for KERNEL


### Surfaces Aligned

#### Surfaces Aligned - 0.3 SOAP

In [226]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index, 1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1013:  10%|███████████                                                                                                   | 1011/10000 [00:36<04:00, 37.43it/s, lowest_mse=0.0215, pred_loss=57.8, trigger=0]

The batch_size is now:  32


Epoch: 2021:  20%|██████████████████████▏                                                                                       | 2021/10000 [00:56<02:04, 64.33it/s, lowest_mse=0.0133, pred_loss=45.5, trigger=0]

The batch_size is now:  64


Epoch: 3031:  30%|█████████████████████████████████▏                                                                            | 3021/10000 [01:09<01:13, 95.48it/s, lowest_mse=0.0113, pred_loss=41.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6037:  60%|█████████████████████████████████████████████████████████████████▏                                          | 6032/10000 [01:34<00:33, 119.18it/s, lowest_mse=0.00898, pred_loss=37.5, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [01:42<00:43, 68.40it/s, lowest_mse=0.00861, pred_loss=36.6, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 36.59 for SOAP
The test error is 49.84 for SOAP


In [227]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos3,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1012:  10%|███████████▏                                                                                                  | 1012/10000 [00:35<03:57, 37.81it/s, lowest_mse=0.0277, pred_loss=73.4, trigger=0]

The batch_size is now:  32


Epoch: 2021:  20%|██████████████████████▏                                                                                       | 2015/10000 [00:55<02:10, 60.99it/s, lowest_mse=0.0186, pred_loss=60.2, trigger=0]

The batch_size is now:  64


Epoch: 3032:  30%|█████████████████████████████████                                                                            | 3030/10000 [01:07<01:07, 103.01it/s, lowest_mse=0.0161, pred_loss=55.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6038:  60%|██████████████████████████████████████████████████████████████████▉                                            | 6033/10000 [01:31<00:31, 125.67it/s, lowest_mse=0.0133, pred_loss=51, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:39<00:42, 70.74it/s, lowest_mse=0.0131, pred_loss=50.4, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 50.43 for SOAP
The test error is 126.2 for SOAP


#### Surfaces Aligned - 0.1 SOAP

In [228]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos1, 
                                                       surface_train_index, surface_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1012:  10%|███████████▏                                                                                                  | 1012/10000 [00:35<03:58, 37.75it/s, lowest_mse=0.0517, pred_loss=66.7, trigger=0]

The batch_size is now:  32


Epoch: 2021:  20%|██████████████████████▏                                                                                       | 2016/10000 [00:55<02:09, 61.68it/s, lowest_mse=0.0381, pred_loss=57.3, trigger=0]

The batch_size is now:  64


Epoch: 3031:  30%|█████████████████████████████████                                                                            | 3031/10000 [01:07<01:08, 102.09it/s, lowest_mse=0.0341, pred_loss=54.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6037:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6031/10000 [01:31<00:32, 123.10it/s, lowest_mse=0.0284, pred_loss=49.5, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:39<00:42, 70.22it/s, lowest_mse=0.0272, pred_loss=48.4, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 48.4 for SOAP
The test error is 62.7 for SOAP


In [229]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos1,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1013:  10%|███████████▏                                                                                                  | 1013/10000 [00:35<03:48, 39.30it/s, lowest_mse=0.0615, pred_loss=79.1, trigger=0]

The batch_size is now:  32


Epoch: 2021:  20%|██████████████████████▏                                                                                       | 2018/10000 [00:54<02:05, 63.67it/s, lowest_mse=0.0488, pred_loss=70.5, trigger=0]

The batch_size is now:  64


Epoch: 3033:  30%|█████████████████████████████████▏                                                                            | 3022/10000 [01:07<01:10, 99.57it/s, lowest_mse=0.0439, pred_loss=66.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6038:  60%|█████████████████████████████████████████████████████████████████▊                                           | 6035/10000 [01:31<00:31, 126.57it/s, lowest_mse=0.0381, pred_loss=62.3, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:39<00:42, 70.35it/s, lowest_mse=0.0374, pred_loss=61.7, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 61.72 for SOAP
The test error is 135.3 for SOAP


#### Surfaces Aligned - 0.3 Kernel

In [230]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 828:   8%|█████████▎                                                                                                       | 826/10000 [00:11<01:40, 91.14it/s, lowest_mse=0.0682, pred_loss=103, trigger=0]

The batch_size is now:  32


Epoch: 1643:  16%|█████████████████▊                                                                                           | 1631/10000 [00:18<01:01, 135.62it/s, lowest_mse=0.0495, pred_loss=87.7, trigger=0]

The batch_size is now:  64


Epoch: 2457:  24%|██████████████████████████▋                                                                                  | 2447/10000 [00:23<00:39, 189.48it/s, lowest_mse=0.0397, pred_loss=78.6, trigger=0]

...

The batch_size is now:  512


Epoch: 4864:  49%|████████████████████████████████████████████████████▉                                                        | 4853/10000 [00:35<00:26, 197.11it/s, lowest_mse=0.0257, pred_loss=63.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:38<00:30, 144.63it/s, lowest_mse=0.0226, pred_loss=59.3, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 59.28 for KERNEL
The test error is 65.53 for KERNEL


In [231]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 829:   8%|█████████▍                                                                                                        | 823/10000 [00:10<01:37, 93.94it/s, lowest_mse=0.056, pred_loss=104, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|██████████████████                                                                                            | 1638/10000 [00:17<00:57, 145.02it/s, lowest_mse=0.042, pred_loss=90.4, trigger=0]

The batch_size is now:  64


Epoch: 2453:  24%|██████████████████████████▌                                                                                  | 2441/10000 [00:22<00:43, 174.86it/s, lowest_mse=0.0347, pred_loss=82.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4873:  48%|████████████████████████████████████████████████████▊                                                        | 4850/10000 [00:33<00:21, 240.95it/s, lowest_mse=0.0234, pred_loss=67.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:36<00:28, 153.53it/s, lowest_mse=0.0213, pred_loss=64.4, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 64.36 for KERNEL
The test error is 164.2 for KERNEL


#### Surfaces Aligned - 0.1 Kernel

In [232]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos1,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 828:   8%|█████████▍                                                                                                        | 826/10000 [00:11<01:38, 93.02it/s, lowest_mse=0.119, pred_loss=101, trigger=0]

The batch_size is now:  32


Epoch: 1642:  16%|█████████████████▊                                                                                           | 1639/10000 [00:18<00:59, 140.05it/s, lowest_mse=0.0975, pred_loss=91.6, trigger=0]

The batch_size is now:  64


Epoch: 2456:  24%|██████████████████████████▋                                                                                  | 2445/10000 [00:22<00:41, 181.80it/s, lowest_mse=0.0864, pred_loss=86.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4869:  49%|████████████████████████████████████████████████████▉                                                        | 4851/10000 [00:34<00:23, 221.26it/s, lowest_mse=0.0664, pred_loss=75.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:37<00:29, 148.86it/s, lowest_mse=0.0613, pred_loss=72.7, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 72.67 for KERNEL
The test error is 78.06 for KERNEL


In [233]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-1, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 829:   8%|█████████▎                                                                                                       | 823/10000 [00:10<01:38, 93.61it/s, lowest_mse=0.0575, pred_loss=106, trigger=0]

The batch_size is now:  32


Epoch: 1644:  16%|█████████████████▊                                                                                           | 1638/10000 [00:17<00:57, 144.56it/s, lowest_mse=0.0424, pred_loss=90.8, trigger=0]

The batch_size is now:  64


Epoch: 2460:  24%|██████████████████████████▌                                                                                  | 2438/10000 [00:22<00:38, 195.25it/s, lowest_mse=0.0349, pred_loss=82.4, trigger=0]

...

The batch_size is now:  512


Epoch: 4863:  48%|████████████████████████████████████████████████████▊                                                        | 4846/10000 [00:32<00:25, 200.36it/s, lowest_mse=0.0241, pred_loss=68.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:36<00:28, 153.83it/s, lowest_mse=0.0216, pred_loss=64.8, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 64.83 for KERNEL
The test error is 165.6 for KERNEL


### Bulk

#### Bulk - 0.3 SOAP

In [76]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos3,
                                                       bulk_train_index, bulk_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [05:50<37:18,  4.02it/s, lowest_mse=0.0085, pred_loss=24.8, trigger=0]

The batch_size is now:  32


Epoch: 2007:  20%|█████████████████████▉                                                                                       | 2007/10000 [08:30<14:57,  8.90it/s, lowest_mse=0.00711, pred_loss=22.1, trigger=0]

The batch_size is now:  64


Epoch: 3011:  30%|████████████████████████████████▊                                                                            | 3010/10000 [09:55<08:04, 14.43it/s, lowest_mse=0.00647, pred_loss=20.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6022:  60%|██████████████████████████████████████████████████████████████████▊                                            | 6022/10000 [11:46<01:38, 40.49it/s, lowest_mse=0.00574, pred_loss=19, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [12:06<05:09,  9.66it/s, lowest_mse=0.0056, pred_loss=18.8, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 18.76 for SOAP
The test error is 20.88 for SOAP


In [77]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos3,
                                                       bulk_biased_train_index, bulk_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|██████████▉                                                                                                  | 1004/10000 [04:54<36:37,  4.09it/s, lowest_mse=0.00885, pred_loss=24.5, trigger=0]

The batch_size is now:  32


Epoch: 2007:  20%|█████████████████████▉                                                                                       | 2007/10000 [07:29<15:26,  8.62it/s, lowest_mse=0.00739, pred_loss=22.1, trigger=0]

The batch_size is now:  64


Epoch: 3011:  30%|████████████████████████████████▊                                                                            | 3010/10000 [08:52<08:09, 14.29it/s, lowest_mse=0.00672, pred_loss=20.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6023:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6022/10000 [10:37<01:33, 42.57it/s, lowest_mse=0.00595, pred_loss=18.8, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [10:57<04:39, 10.67it/s, lowest_mse=0.00581, pred_loss=18.6, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 18.62 for SOAP
The test error is 23.92 for SOAP


#### Bulk - 0.1 Soap

In [78]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos1,
                                                       bulk_train_index, bulk_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████▏                                                                                                   | 1005/10000 [02:38<18:39,  8.04it/s, lowest_mse=0.042, pred_loss=43.2, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2009/10000 [04:05<08:59, 14.81it/s, lowest_mse=0.0385, pred_loss=40.9, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████▏                                                                            | 3012/10000 [04:54<04:31, 25.78it/s, lowest_mse=0.0368, pred_loss=39.7, trigger=0]

...

The batch_size is now:  512


Epoch: 6024:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6024/10000 [06:09<01:15, 52.73it/s, lowest_mse=0.0346, pred_loss=38.3, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|██████████████████████████████████████████████████████████████████████████████▌                                 | 7015/10000 [06:27<02:44, 18.10it/s, lowest_mse=0.0342, pred_loss=38, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 38.03 for SOAP
The test error is 40.9 for SOAP


In [79]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos1,
                                                       bulk_biased_train_index, bulk_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [05:01<40:11,  3.73it/s, lowest_mse=0.0445, pred_loss=42.3, trigger=0]

The batch_size is now:  32


Epoch: 2007:  20%|██████████████████████                                                                                        | 2007/10000 [08:03<17:53,  7.45it/s, lowest_mse=0.0408, pred_loss=41.2, trigger=0]

The batch_size is now:  64


Epoch: 3011:  30%|█████████████████████████████████▍                                                                             | 3010/10000 [09:29<08:31, 13.68it/s, lowest_mse=0.039, pred_loss=39.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6022:  60%|██████████████████████████████████████████████████████████████████▏                                           | 6020/10000 [11:20<01:42, 38.94it/s, lowest_mse=0.0367, pred_loss=38.4, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [11:42<04:58,  9.99it/s, lowest_mse=0.0362, pred_loss=38.2, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 38.16 for SOAP
The test error is 44.47 for SOAP


#### Bulk - 0.3 Kernel

In [80]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_200, bulk_dos3,
                                                              bulk_train_index, bulk_test_index,
                                                              bulk_kMM_200, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:27<12:58, 11.81it/s, lowest_mse=0.0344, pred_loss=67.9, trigger=0]

The batch_size is now:  32


Epoch: 1614:  16%|█████████████████▋                                                                                            | 1612/10000 [02:14<07:02, 19.85it/s, lowest_mse=0.0272, pred_loss=53.7, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▌                                                                                   | 2418/10000 [02:43<03:38, 34.71it/s, lowest_mse=0.0251, pred_loss=54.2, trigger=0]

...

The batch_size is now:  512


Epoch: 4845:  48%|█████████████████████████████████████████████████████▏                                                        | 4838/10000 [03:22<00:56, 91.10it/s, lowest_mse=0.0238, pred_loss=47.2, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:30<02:43, 26.79it/s, lowest_mse=0.0236, pred_loss=38.5, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 38.53 for KERNEL
The test error is 39.54 for KERNEL


In [115]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_100, bulk_dos3,
                                                              bulk_biased_train_index, bulk_biased_test_index,
                                                              bulk_kMM_100, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████                                                                                                       | 809/10000 [01:05<09:50, 15.55it/s, lowest_mse=0.0369, pred_loss=65.9, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1614/10000 [01:41<05:01, 27.77it/s, lowest_mse=0.0292, pred_loss=46.8, trigger=0]

The batch_size is now:  64


Epoch: 2426:  24%|███████████████████████████▏                                                                                    | 2422/10000 [02:01<02:35, 48.73it/s, lowest_mse=0.0264, pred_loss=42, trigger=0]

...

The batch_size is now:  512


Epoch: 4849:  48%|████████████████████████████████████████████████████▊                                                        | 4848/10000 [02:32<00:45, 112.47it/s, lowest_mse=0.0242, pred_loss=39.7, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|██████████████████████████████████████████████████████████████▍                                                | 5629/10000 [02:38<02:03, 35.42it/s, lowest_mse=0.024, pred_loss=37.8, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 37.8 for KERNEL
The test error is 49.49 for KERNEL


#### Bulk - 0.1 Kernel

In [116]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_200, bulk_dos1,
                                                              bulk_train_index, bulk_test_index,
                                                              bulk_kMM_200, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:24<12:57, 11.82it/s, lowest_mse=0.0873, pred_loss=85.1, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:09<06:03, 23.05it/s, lowest_mse=0.0775, pred_loss=63.9, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2419/10000 [02:36<03:16, 38.59it/s, lowest_mse=0.0739, pred_loss=63.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4841:  48%|█████████████████████████████████████████████████████▏                                                        | 4839/10000 [03:17<01:22, 62.64it/s, lowest_mse=0.0712, pred_loss=61.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:27<02:40, 27.17it/s, lowest_mse=0.0708, pred_loss=54.7, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 54.68 for KERNEL
The test error is 56.12 for KERNEL


In [117]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_100, bulk_dos1,
                                                              bulk_biased_train_index, bulk_biased_test_index,
                                                              bulk_kMM_100, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████▏                                                                                                       | 809/10000 [01:04<09:31, 16.08it/s, lowest_mse=0.092, pred_loss=68.1, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1614/10000 [01:39<04:58, 28.06it/s, lowest_mse=0.0818, pred_loss=61.2, trigger=0]

The batch_size is now:  64


Epoch: 2426:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:00<02:35, 48.88it/s, lowest_mse=0.0779, pred_loss=57.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4853:  48%|█████████████████████████████████████████████████████▏                                                        | 4840/10000 [02:29<00:44, 117.18it/s, lowest_mse=0.074, pred_loss=55.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:34<02:00, 36.32it/s, lowest_mse=0.0734, pred_loss=54.3, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 54.33 for KERNEL
The test error is 65.97 for KERNEL


### Total

#### Total - 0.3 Soap


In [118]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:22<31:42,  4.73it/s, lowest_mse=0.0172, pred_loss=28.8, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████▍                                                                                         | 2007/10000 [05:10<11:20, 11.74it/s, lowest_mse=0.0153, pred_loss=27, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [06:10<05:26, 21.43it/s, lowest_mse=0.0144, pred_loss=25.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6028:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6025/10000 [07:24<01:00, 65.67it/s, lowest_mse=0.0131, pred_loss=24.5, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:39<03:15, 15.28it/s, lowest_mse=0.0129, pred_loss=24.1, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 24.1 for SOAP
The test error is 25.9 for SOAP


In [449]:
#CHANGED
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|██████████▉                                                                                                  | 1004/10000 [03:57<28:34,  5.25it/s, lowest_mse=0.00955, pred_loss=27.4, trigger=0]

The batch_size is now:  32


Epoch: 1671:  17%|██████████████████▏                                                                                          | 1671/10000 [05:16<26:15,  5.29it/s, lowest_mse=0.00855, pred_loss=24.7, trigger=0]


KeyboardInterrupt: 

In [119]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:21<37:27,  4.00it/s, lowest_mse=0.0178, pred_loss=27.9, trigger=0]

The batch_size is now:  32


Epoch: 2007:  20%|██████████████████████                                                                                        | 2007/10000 [05:57<16:26,  8.10it/s, lowest_mse=0.0157, pred_loss=26.2, trigger=0]

The batch_size is now:  64


Epoch: 3010:  30%|█████████████████████████████████                                                                             | 3010/10000 [07:22<08:26, 13.81it/s, lowest_mse=0.0147, pred_loss=25.4, trigger=0]

...

The batch_size is now:  512


Epoch: 6023:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6023/10000 [09:11<01:31, 43.69it/s, lowest_mse=0.0135, pred_loss=24.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [09:31<04:03, 12.28it/s, lowest_mse=0.0132, pred_loss=23.5, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 23.54 for SOAP
The test error is 30.39 for SOAP


In [120]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████▏                                                                                                   | 1004/10000 [04:27<26:09,  5.73it/s, lowest_mse=0.014, pred_loss=26.2, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [06:21<12:09, 10.96it/s, lowest_mse=0.0123, pred_loss=24.2, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [07:25<06:18, 18.47it/s, lowest_mse=0.0115, pred_loss=23.4, trigger=0]

...

The batch_size is now:  512


Epoch: 6027:  60%|██████████████████████████████████████████████████████████████████▏                                           | 6021/10000 [08:46<01:08, 57.76it/s, lowest_mse=0.0104, pred_loss=22.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [09:02<03:50, 12.93it/s, lowest_mse=0.0103, pred_loss=21.7, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 21.73 for SOAP
The test error is 36.76 for SOAP


#### Total - 0.1 Soap

In [121]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:14<26:08,  5.73it/s, lowest_mse=0.0527, pred_loss=42.8, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [04:57<10:51, 12.26it/s, lowest_mse=0.0487, pred_loss=40.6, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [05:54<05:32, 21.04it/s, lowest_mse=0.0467, pred_loss=39.4, trigger=0]

...

The batch_size is now:  512


Epoch: 6024:  60%|██████████████████████████████████████████████████████████████████▏                                           | 6021/10000 [07:10<01:21, 48.54it/s, lowest_mse=0.0441, pred_loss=38.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:24<03:08, 15.80it/s, lowest_mse=0.0436, pred_loss=37.9, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 37.87 for SOAP
The test error is 40.09 for SOAP


In [122]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:20<25:00,  5.99it/s, lowest_mse=0.0553, pred_loss=42.2, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [05:04<10:56, 12.18it/s, lowest_mse=0.0511, pred_loss=40.1, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████▍                                                                             | 3011/10000 [06:00<05:13, 22.28it/s, lowest_mse=0.049, pred_loss=39.1, trigger=0]

...

The batch_size is now:  512


Epoch: 6027:  60%|██████████████████████████████████████████████████████████████████▏                                           | 6021/10000 [07:13<01:05, 60.39it/s, lowest_mse=0.0462, pred_loss=37.8, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:26<03:10, 15.69it/s, lowest_mse=0.0457, pred_loss=37.5, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 37.48 for SOAP
The test error is 44.07 for SOAP


In [123]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████▏                                                                                                   | 1004/10000 [03:14<24:32,  6.11it/s, lowest_mse=0.049, pred_loss=41.1, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████▍                                                                                         | 2007/10000 [05:02<10:47, 12.34it/s, lowest_mse=0.0452, pred_loss=39, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████                                                                             | 3011/10000 [05:59<05:19, 21.89it/s, lowest_mse=0.0433, pred_loss=38.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6027:  60%|███████████████████████████████████████████████████████████████████▍                                            | 6022/10000 [07:12<01:05, 60.54it/s, lowest_mse=0.0409, pred_loss=37, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:26<03:09, 15.72it/s, lowest_mse=0.0405, pred_loss=36.7, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 36.72 for SOAP
The test error is 47.14 for SOAP


#### Total - 0.3 Kernel

In [124]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos3,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:29<13:16, 11.54it/s, lowest_mse=0.0472, pred_loss=60.1, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:17<06:35, 21.20it/s, lowest_mse=0.0392, pred_loss=47.8, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2420/10000 [02:45<03:24, 37.09it/s, lowest_mse=0.0365, pred_loss=44.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4848:  48%|█████████████████████████████████████████████████████▊                                                         | 4844/10000 [03:22<00:50, 102.24it/s, lowest_mse=0.0345, pred_loss=55, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:29<02:42, 26.88it/s, lowest_mse=0.0343, pred_loss=39.3, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 39.28 for KERNEL
The test error is 40.15 for KERNEL


In [125]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos3,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:29<13:16, 11.54it/s, lowest_mse=0.0486, pred_loss=55.7, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:18<06:35, 21.19it/s, lowest_mse=0.0403, pred_loss=47.1, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2418/10000 [02:44<03:34, 35.35it/s, lowest_mse=0.0377, pred_loss=43.5, trigger=0]

...

The batch_size is now:  512


Epoch: 4847:  48%|█████████████████████████████████████████████████████▎                                                        | 4842/10000 [03:19<00:51, 99.41it/s, lowest_mse=0.0357, pred_loss=53.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:26<02:40, 27.29it/s, lowest_mse=0.0354, pred_loss=38.5, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 38.53 for KERNEL
The test error is 47.84 for KERNEL


In [126]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_dos3,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:17<12:43, 12.04it/s, lowest_mse=0.0448, pred_loss=49.7, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [01:59<05:50, 23.93it/s, lowest_mse=0.0366, pred_loss=43.5, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2419/10000 [02:24<03:19, 37.97it/s, lowest_mse=0.0338, pred_loss=43.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4848:  48%|████████████████████████████████████████████████████▊                                                        | 4843/10000 [02:57<00:50, 102.12it/s, lowest_mse=0.0315, pred_loss=43.7, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:04<02:22, 30.57it/s, lowest_mse=0.0311, pred_loss=37.9, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 37.85 for KERNEL
The test error is 47.54 for KERNEL


#### Total - 0.1 Kernel 

In [127]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos1,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████▎                                                                                                         | 807/10000 [01:30<13:25, 11.41it/s, lowest_mse=0.102, pred_loss=73, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:19<06:34, 21.27it/s, lowest_mse=0.0916, pred_loss=59.7, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▋                                                                                   | 2421/10000 [02:47<03:29, 36.10it/s, lowest_mse=0.0874, pred_loss=56.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4847:  48%|█████████████████████████████████████████████████████▎                                                        | 4842/10000 [03:26<00:54, 95.27it/s, lowest_mse=0.0834, pred_loss=59.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:33<02:45, 26.40it/s, lowest_mse=0.0828, pred_loss=52.2, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 52.15 for KERNEL
The test error is 53.23 for KERNEL


In [139]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos1,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                        | 807/10000 [01:28<13:18, 11.51it/s, lowest_mse=0.106, pred_loss=72.1, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:16<06:37, 21.12it/s, lowest_mse=0.0946, pred_loss=58.7, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|███████████████████████████                                                                                     | 2420/10000 [02:43<03:27, 36.59it/s, lowest_mse=0.0902, pred_loss=58, trigger=0]

...

The batch_size is now:  512


Epoch: 4847:  48%|█████████████████████████████████████████████████████▎                                                        | 4846/10000 [03:20<00:52, 97.83it/s, lowest_mse=0.0862, pred_loss=58.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:28<02:42, 26.98it/s, lowest_mse=0.0856, pred_loss=51.3, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 51.31 for KERNEL
The test error is 60.57 for KERNEL


In [140]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_dos1,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████▏                                                                                                        | 808/10000 [01:21<12:20, 12.42it/s, lowest_mse=0.0999, pred_loss=62, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [02:06<05:48, 24.03it/s, lowest_mse=0.0892, pred_loss=58.8, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▋                                                                                   | 2423/10000 [02:32<03:01, 41.83it/s, lowest_mse=0.0849, pred_loss=56.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4850:  48%|█████████████████████████████████████████████████████▊                                                         | 4848/10000 [03:05<00:46, 110.54it/s, lowest_mse=0.0807, pred_loss=54, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|███████████████████████████████████████████████████████████████                                                 | 5629/10000 [03:11<02:29, 29.33it/s, lowest_mse=0.08, pred_loss=51.6, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 51.6 for KERNEL
The test error is 57.83 for KERNEL


### Total Aligned

#### Total Aligned - 0.3 Soap


In [141]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████▏                                                                                                   | 1004/10000 [03:09<24:19,  6.16it/s, lowest_mse=0.015, pred_loss=27.9, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [04:53<10:48, 12.33it/s, lowest_mse=0.0133, pred_loss=25.2, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████▋                                                                              | 3010/10000 [05:52<05:49, 19.97it/s, lowest_mse=0.0124, pred_loss=24, trigger=0]

...

The batch_size is now:  512


Epoch: 6027:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6025/10000 [07:11<01:03, 63.00it/s, lowest_mse=0.0113, pred_loss=22.8, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:26<03:09, 15.72it/s, lowest_mse=0.0111, pred_loss=22.4, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 22.36 for SOAP
The test error is 24.29 for SOAP


In [142]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:09<24:18,  6.17it/s, lowest_mse=0.0155, pred_loss=26.3, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [04:53<10:47, 12.34it/s, lowest_mse=0.0136, pred_loss=24.4, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [05:51<05:34, 20.92it/s, lowest_mse=0.0127, pred_loss=23.6, trigger=0]

...

The batch_size is now:  512


Epoch: 6027:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6025/10000 [07:08<01:03, 62.88it/s, lowest_mse=0.0116, pred_loss=22.3, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:22<03:08, 15.87it/s, lowest_mse=0.0114, pred_loss=21.8, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 21.84 for SOAP
The test error is 28.61 for SOAP


In [143]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:14<24:55,  6.02it/s, lowest_mse=0.0129, pred_loss=25.2, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [05:00<10:51, 12.27it/s, lowest_mse=0.0112, pred_loss=23.2, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████▏                                                                            | 3013/10000 [05:58<04:58, 23.44it/s, lowest_mse=0.0104, pred_loss=22.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6028:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6021/10000 [07:12<01:03, 62.54it/s, lowest_mse=0.00946, pred_loss=21.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:26<03:10, 15.70it/s, lowest_mse=0.0093, pred_loss=20.7, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 20.67 for SOAP
The test error is 32.52 for SOAP


#### Total Aligned - 0.1 Soap

In [144]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:14<25:49,  5.81it/s, lowest_mse=0.0505, pred_loss=41.3, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [05:04<11:27, 11.63it/s, lowest_mse=0.0467, pred_loss=39.5, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [06:04<05:50, 19.95it/s, lowest_mse=0.0447, pred_loss=38.6, trigger=0]

...

The batch_size is now:  512


Epoch: 6028:  60%|██████████████████████████████████████████████████████████████████▏                                           | 6021/10000 [07:19<01:02, 63.87it/s, lowest_mse=0.0422, pred_loss=37.4, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|██████████████████████████████████████████████████████████████████████████████▌                                 | 7015/10000 [07:35<03:13, 15.41it/s, lowest_mse=0.0418, pred_loss=37, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 37.04 for SOAP
The test error is 39.44 for SOAP


In [145]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:12<24:24,  6.14it/s, lowest_mse=0.0532, pred_loss=41.1, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████▎                                                                                        | 2007/10000 [04:57<11:01, 12.08it/s, lowest_mse=0.049, pred_loss=39.3, trigger=0]

The batch_size is now:  64


Epoch: 3012:  30%|█████████████████████████████████                                                                             | 3010/10000 [05:55<05:37, 20.72it/s, lowest_mse=0.0469, pred_loss=38.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6028:  60%|███████████████████████████████████████████████████████████████████▍                                            | 6024/10000 [07:08<01:01, 64.73it/s, lowest_mse=0.0443, pred_loss=37, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:21<03:08, 15.88it/s, lowest_mse=0.0438, pred_loss=36.7, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 36.69 for SOAP
The test error is 43.51 for SOAP


In [146]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|███████████                                                                                                   | 1004/10000 [03:16<24:56,  6.01it/s, lowest_mse=0.0481, pred_loss=40.6, trigger=0]

The batch_size is now:  32


Epoch: 2008:  20%|██████████████████████                                                                                        | 2007/10000 [05:00<10:52, 12.25it/s, lowest_mse=0.0443, pred_loss=38.8, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████                                                                             | 3011/10000 [05:56<05:18, 21.95it/s, lowest_mse=0.0425, pred_loss=37.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6028:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6024/10000 [07:10<01:01, 64.29it/s, lowest_mse=0.0401, pred_loss=36.6, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [07:23<03:08, 15.83it/s, lowest_mse=0.0397, pred_loss=36.3, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 36.32 for SOAP
The test error is 44.49 for SOAP


#### Total Aligned - 0.3 Kernel

In [147]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos3,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:30<13:16, 11.54it/s, lowest_mse=0.0445, pred_loss=56.3, trigger=0]

The batch_size is now:  32


Epoch: 1614:  16%|█████████████████▋                                                                                            | 1612/10000 [02:21<14:11,  9.85it/s, lowest_mse=0.0364, pred_loss=45.8, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▋                                                                                   | 2423/10000 [02:48<03:14, 38.91it/s, lowest_mse=0.0337, pred_loss=43.5, trigger=0]

...

The batch_size is now:  512


Epoch: 4848:  48%|████████████████████████████████████████████████████▊                                                        | 4847/10000 [03:24<00:50, 102.53it/s, lowest_mse=0.0316, pred_loss=53.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:31<02:43, 26.67it/s, lowest_mse=0.0313, pred_loss=37.6, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 37.59 for KERNEL
The test error is 38.46 for KERNEL


In [148]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos3,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:28<13:08, 11.66it/s, lowest_mse=0.0449, pred_loss=57.2, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:15<06:32, 21.39it/s, lowest_mse=0.0368, pred_loss=44.2, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:41<03:19, 37.92it/s, lowest_mse=0.0343, pred_loss=40.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4847:  48%|██████████████████████████████████████████████████████▎                                                         | 4844/10000 [03:17<00:53, 96.37it/s, lowest_mse=0.0324, pred_loss=52, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:24<02:38, 27.52it/s, lowest_mse=0.0321, pred_loss=36.7, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 36.69 for KERNEL
The test error is 46.22 for KERNEL


In [149]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_aligned_dos3,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:20<12:11, 12.56it/s, lowest_mse=0.0432, pred_loss=50.8, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [02:04<05:45, 24.25it/s, lowest_mse=0.0349, pred_loss=42.6, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▌                                                                                   | 2420/10000 [02:30<03:12, 39.46it/s, lowest_mse=0.0321, pred_loss=40.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4850:  48%|████████████████████████████████████████████████████▋                                                        | 4838/10000 [03:03<00:48, 107.45it/s, lowest_mse=0.0297, pred_loss=41.2, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:09<02:27, 29.68it/s, lowest_mse=0.0293, pred_loss=36.7, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 36.71 for KERNEL
The test error is 43.37 for KERNEL


#### Total Aligned - 0.1 Kernel 

In [150]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos1,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                        | 807/10000 [01:30<13:21, 11.47it/s, lowest_mse=0.099, pred_loss=67.9, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:19<06:38, 21.05it/s, lowest_mse=0.0886, pred_loss=61.1, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2418/10000 [02:45<03:36, 35.09it/s, lowest_mse=0.0844, pred_loss=58.5, trigger=0]

...

The batch_size is now:  512


Epoch: 4848:  48%|█████████████████████████████████████████████████████▏                                                        | 4838/10000 [03:21<00:52, 99.16it/s, lowest_mse=0.0805, pred_loss=58.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:28<02:41, 27.05it/s, lowest_mse=0.0799, pred_loss=51.2, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 51.2 for KERNEL
The test error is 52.35 for KERNEL


In [151]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos1,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                        | 807/10000 [01:29<13:38, 11.23it/s, lowest_mse=0.102, pred_loss=76.4, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:18<06:38, 21.05it/s, lowest_mse=0.0915, pred_loss=57.7, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▌                                                                                   | 2418/10000 [02:46<03:39, 34.58it/s, lowest_mse=0.0873, pred_loss=55.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4848:  48%|████████████████████████████████████████████████████▊                                                        | 4844/10000 [03:25<00:50, 101.62it/s, lowest_mse=0.0834, pred_loss=57.2, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:32<02:44, 26.55it/s, lowest_mse=0.0828, pred_loss=50.4, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 50.45 for KERNEL
The test error is 60.13 for KERNEL


In [152]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_aligned_dos1,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:21<13:00, 11.77it/s, lowest_mse=0.0987, pred_loss=61.5, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:05<06:08, 22.79it/s, lowest_mse=0.0879, pred_loss=57.2, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:30<03:04, 41.16it/s, lowest_mse=0.0836, pred_loss=54.7, trigger=0]

...

The batch_size is now:  512


Epoch: 4846:  48%|█████████████████████████████████████████████████████▏                                                        | 4840/10000 [03:04<00:52, 97.82it/s, lowest_mse=0.0792, pred_loss=54.7, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:11<02:28, 29.42it/s, lowest_mse=0.0786, pred_loss=51.1, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 51.07 for KERNEL
The test error is 55.2 for KERNEL


## Baseline GD Model - LBFGS

### Surfaces

#### Surfaces - 0.3 SOAP 

In [240]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos3, surface_train_index, surface_test_index,
                                                            1e-2, 6, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.88it/s, lowest_mse=0.00555, pred_loss=28.2, trigger=0]

LBFGS Unbiased
The train error is 28.21 for SOAP
The test error is 44.11 for SOAP


In [245]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.01it/s, lowest_mse=0.0131, pred_loss=49.4, trigger=0]

LBFGS holdout
The train error is 48.58 for SOAP
The test error is 124.2 for SOAP


#### Surfaces - 0.1 SOAP 

In [246]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos1,
                                                            surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.65it/s, lowest_mse=0.0187, pred_loss=39.1, trigger=0]

LBFGS Unbiased
The train error is 39.02 for SOAP
The test error is 58.14 for SOAP


In [247]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.16it/s, lowest_mse=0.0361, pred_loss=60, trigger=0]

LBFGS holdout
The train error is 59.09 for SOAP
The test error is 130.2 for SOAP


#### Surfaces - 0.3 Kernel

In [248]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos3,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.07it/s, lowest_mse=0.00915, pred_loss=36.2, trigger=0]

LBFGS Unbiased
The train error is 36.22 for Kernel
The test error is 50.28 for Kernel


In [249]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos3,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.96it/s, lowest_mse=0.00763, pred_loss=37, trigger=0]

LBFGS Holdout
The train error is 37.04 for Kernel
The test error is 125.8 for Kernel


#### Surfaces - 0.1 Kernel

In [250]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos1,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.76it/s, lowest_mse=0.0233, pred_loss=43.6, trigger=0]

LBFGS Unbiased
The train error is 43.58 for Kernel
The test error is 57.74 for Kernel


In [251]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos1,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.16it/s, lowest_mse=0.0257, pred_loss=49.8, trigger=0]

LBFGS Holdout
The train error is 49.83 for Kernel
The test error is 132.5 for Kernel


### Surfaces Aligned

#### Surfaces Aligned - 0.3 SOAP 

In [252]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.82it/s, lowest_mse=0.00579, pred_loss=30, trigger=0]

LBFGS Unbiased
The train error is 30.01 for SOAP
The test error is 45.27 for SOAP


In [253]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.34it/s, lowest_mse=0.0128, pred_loss=49.8, trigger=0]

LBFGS holdout
The train error is 49.8 for SOAP
The test error is 123.3 for SOAP


#### Surfaces Aligned - 0.1 SOAP 

In [254]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos1,
                                                            surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.94it/s, lowest_mse=0.0186, pred_loss=40.1, trigger=0]

LBFGS Unbiased
The train error is 40.03 for SOAP
The test error is 59.22 for SOAP


In [255]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.58it/s, lowest_mse=0.0356, pred_loss=60.8, trigger=0]

LBFGS holdout
The train error is 60.18 for SOAP
The test error is 127.8 for SOAP


#### Surfaces Aligned - 0.3 Kernel

In [256]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos3,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.12it/s, lowest_mse=0.00821, pred_loss=35.7, trigger=0]

LBFGS Unbiased
The train error is 35.74 for Kernel
The test error is 47.47 for Kernel


In [257]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos3,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.34it/s, lowest_mse=0.00766, pred_loss=38.6, trigger=0]

LBFGS Holdout
The train error is 38.61 for Kernel
The test error is 122.2 for Kernel


#### Surfaces Aligned - 0.1 Kernel

In [258]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos1,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.89it/s, lowest_mse=0.0253, pred_loss=46.7, trigger=0]

LBFGS Unbiased
The train error is 46.72 for Kernel
The test error is 60.85 for Kernel


In [259]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos1,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.63it/s, lowest_mse=0.0251, pred_loss=50.5, trigger=0]

LBFGS Holdout
The train error is 50.51 for Kernel
The test error is 132.9 for Kernel


### Bulk

#### Bulk - 0.3 SOAP

In [263]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.39s/it, lowest_mse=0.00409, pred_loss=16, trigger=0]

LBFGS Unbiased
The train error is 15.77 for SOAP
The test error is 20.84 for SOAP


In [264]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.33s/it, lowest_mse=0.00451, pred_loss=16.4, trigger=0]

LBFGS Biased
The train error is 16.41 for SOAP
The test error is 23.16 for SOAP


#### Bulk - 0.1 SOAP

In [265]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:21<00:00,  1.40s/it, lowest_mse=0.0277, pred_loss=34.2, trigger=0]

LBFGS Unbiased
The train error is 34.08 for SOAP
The test error is 40.3 for SOAP


In [266]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.36s/it, lowest_mse=0.029, pred_loss=34.1, trigger=0]

LBFGS Biased
The train error is 34.06 for SOAP
The test error is 44.22 for SOAP


#### Bulk - 0.3 Kernel

In [267]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_200, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            bulk_kMM_200, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.77it/s, lowest_mse=0.00994, pred_loss=25, trigger=0]

LBFGS Unbiased
The train error is 25.0 for Kernel
The test error is 26.22 for Kernel


In [268]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_100, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            bulk_kMM_100, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.85it/s, lowest_mse=0.00899, pred_loss=23.2, trigger=0]

LBFGS Biased
The train error is 23.16 for Kernel
The test error is 27.1 for Kernel


#### Bulk - 0.1 Kernel

In [269]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_200, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            bulk_kMM_200, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:11<00:00,  1.28it/s, lowest_mse=0.0406, pred_loss=41.4, trigger=0]

LBFGS Unbiased
The train error is 41.43 for Kernel
The test error is 43.47 for Kernel


In [270]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_100, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            bulk_kMM_100, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.87it/s, lowest_mse=0.0459, pred_loss=42.9, trigger=0]

LBFGS Biased
The train error is 42.94 for Kernel
The test error is 48.86 for Kernel


### Total

#### Total - 0.3 SOAP

In [271]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.32s/it, lowest_mse=0.00966, pred_loss=20.9, trigger=0]

LBFGS Unbiased
The train error is 20.82 for SOAP
The test error is 23.87 for SOAP


In [272]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.28s/it, lowest_mse=0.00978, pred_loss=20.2, trigger=0]

LBFGS Biased
The train error is 20.23 for SOAP
The test error is 28.22 for SOAP


In [273]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.30s/it, lowest_mse=0.00776, pred_loss=18.9, trigger=0]

LBFGS Holdout
The train error is 18.61 for SOAP
The test error is 34.42 for SOAP


#### Total - 0.1 SOAP

In [274]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:19<00:00,  1.32s/it, lowest_mse=0.0352, pred_loss=34, trigger=0]

LBFGS Unbiased
The train error is 33.95 for SOAP
The test error is 38.03 for SOAP


In [275]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.36s/it, lowest_mse=0.0366, pred_loss=33.5, trigger=0]

LBFGS Biased
The train error is 33.4 for SOAP
The test error is 42.61 for SOAP


In [276]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:22<00:00,  1.51s/it, lowest_mse=0.0335, pred_loss=33.4, trigger=0]

LBFGS Holdout
The train error is 33.23 for SOAP
The test error is 44.89 for SOAP


#### Total - 0.3 Kernel 

In [277]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos3,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.85it/s, lowest_mse=0.0165, pred_loss=27.2, trigger=0]

LBFGS Unbiased
The train error is 27.25 for Kernel
The test error is 29.13 for Kernel


In [278]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:09<00:00,  1.55it/s, lowest_mse=0.017, pred_loss=26.7, trigger=0]

LBFGS Biased
The train error is 26.69 for Kernel
The test error is 34.31 for Kernel


In [279]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_100, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_100, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.40it/s, lowest_mse=0.0185, pred_loss=29.2, trigger=0]

LBFGS Holdout
The train error is 29.21 for Kernel
The test error is 41.58 for Kernel


#### Total - 0.1 Kernel

In [280]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  1.89it/s, lowest_mse=0.0517, pred_loss=41.2, trigger=0]

LBFGS Unbiased
The train error is 41.2 for Kernel
The test error is 43.15 for Kernel


In [281]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.72it/s, lowest_mse=0.0517, pred_loss=39.9, trigger=0]

LBFGS Biased
The train error is 39.88 for Kernel
The test error is 46.84 for Kernel


In [282]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.68it/s, lowest_mse=0.0491, pred_loss=40.4, trigger=0]

LBFGS Holdout
The train error is 40.4 for Kernel
The test error is 49.56 for Kernel


### Total Aligned

#### Total Aligned - 0.3 SOAP

In [283]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.35s/it, lowest_mse=0.00843, pred_loss=19.5, trigger=0]

LBFGS Unbiased
The train error is 19.34 for SOAP
The test error is 22.58 for SOAP


In [284]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.35s/it, lowest_mse=0.00841, pred_loss=18.8, trigger=0]

LBFGS Biased
The train error is 18.77 for SOAP
The test error is 26.81 for SOAP


In [285]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.39s/it, lowest_mse=0.00713, pred_loss=18.1, trigger=0]

LBFGS Holdout
The train error is 18.0 for SOAP
The test error is 30.51 for SOAP


#### Total Aligned - 0.1 SOAP

In [286]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-2, 45, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 44: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:02<00:00,  1.39s/it, lowest_mse=0.0318, pred_loss=32.3, trigger=0]

LBFGS Unbiased
The train error is 32.31 for SOAP
The test error is 37.76 for SOAP


In [294]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:20<00:00,  1.37s/it, lowest_mse=0.036, pred_loss=33.3, trigger=0]

LBFGS Biased
The train error is 32.91 for SOAP
The test error is 41.76 for SOAP


In [287]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:21<00:00,  1.44s/it, lowest_mse=0.0325, pred_loss=32.8, trigger=0]

LBFGS Holdout
The train error is 32.55 for SOAP
The test error is 42.52 for SOAP


#### Total Aligned - 0.3 Kernel 

In [288]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.36it/s, lowest_mse=0.0163, pred_loss=27.1, trigger=0]

LBFGS Unbiased
The train error is 27.1 for Kernel
The test error is 28.44 for Kernel


In [289]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.79it/s, lowest_mse=0.0158, pred_loss=25.7, trigger=0]

LBFGS Biased
The train error is 25.74 for Kernel
The test error is 32.59 for Kernel


In [290]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_100, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_100, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.17it/s, lowest_mse=0.0134, pred_loss=24.8, trigger=0]

LBFGS Holdout
The train error is 24.78 for Kernel
The test error is 35.39 for Kernel


#### Total Aligned - 0.1 Kernel

In [291]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.56it/s, lowest_mse=0.053, pred_loss=41.7, trigger=0]

LBFGS Unbiased
The train error is 41.69 for Kernel
The test error is 43.45 for Kernel


In [292]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  1.90it/s, lowest_mse=0.0521, pred_loss=40, trigger=0]

LBFGS Biased
The train error is 40.03 for Kernel
The test error is 46.99 for Kernel


In [293]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:09<00:00,  1.52it/s, lowest_mse=0.0501, pred_loss=40.8, trigger=0]

LBFGS Holdout
The train error is 40.76 for Kernel
The test error is 47.22 for Kernel


## Baseline Linalg Models - Truncated

In [326]:
xdos = xdos2[:cutoff_index]
total_dos3 = total_dos3[:, :cutoff_index]
total_dos1 = total_dos1[:, :cutoff_index]

surface_dos3 = surface_dos3[:, :cutoff_index]
surface_dos1 = surface_dos1[:, :cutoff_index]

surface_aligned_dos3 = surface_aligned_dos3[:, :cutoff_index]
surface_aligned_dos1 = surface_aligned_dos1[:, :cutoff_index]

bulk_dos3 = bulk_dos3[:, :cutoff_index]
bulk_dos1 = bulk_dos1[:, :cutoff_index]

total_aligned_dos3 = total_aligned_dos3[:, :cutoff_index]
total_aligned_dos1 = total_aligned_dos1[:, :cutoff_index]

### Surfaces

#### Surfaces - 0.3 SOAP

In [327]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos3, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos3, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Holdout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 38.57 for SOAP
The test error is 60.68 for SOAP
Holdout:
The train error is 56.29 for SOAP
The test error is 100.3 for SOAP


#### Surfaces - 0.1 SOAP

In [328]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos1, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_dos1, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Goldout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 53.65 for SOAP
The test error is 77.04 for SOAP
Goldout:
The train error is 70.69 for SOAP
The test error is 112.2 for SOAP


#### Surfaces - 0.3 Kernel

In [329]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos3, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            regularization, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 34.35 for n_refs = 30
The test error is 53.55 for n_refs = 30
Biased
The train error is 35.34 for n_refs = 30
The test error is 100.0 for n_refs = 30


#### Surfaces - 0.1 Kernel

In [330]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos1, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 50.45 for n_refs = 30
The test error is 72.96 for n_refs = 30
Biased
The train error is 66.17 for n_refs = 30
The test error is 114.2 for n_refs = 30


### Surfaces Aligned

####  Surfaces Aligned - 0.3 SOAP

In [331]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Holdout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 37.9 for SOAP
The test error is 57.48 for SOAP
Holdout:
The train error is 55.35 for SOAP
The test error is 97.43 for SOAP


####  Surfaces Aligned - 0.1 SOAP

In [332]:
U_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos1, surface_train_index, surface_test_index, 1e-2)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

B_weight_s03, loss_dos, test_loss_dos =normal_reg_train_A(surface_soap, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index, 1e-1)

print ("Goldout:")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 53.07 for SOAP
The test error is 74.84 for SOAP
Goldout:
The train error is 69.99 for SOAP
The test error is 109.9 for SOAP


####  Surfaces Aligned - 0.3 Kernel

In [333]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos3, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            regularization, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 33.97 for n_refs = 30
The test error is 51.68 for n_refs = 30
Biased
The train error is 35.12 for n_refs = 30
The test error is 98.99 for n_refs = 30


####  Surfaces Aligned - 0.1 Kernel

In [334]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos1, surface_train_index, surface_test_index,
                                                            regularization, surface_kMM_30)
print ("Smearing: 0.1")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))
B_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(surface_kernel_30, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, surface_kMM_30)
print ("Biased")
print ("The train error is {:.4} for n_refs = 30".format(loss_dos))
print ("The test error is {:.4} for n_refs = 30".format(test_loss_dos))

Smearing: 0.1
Unbiased
The train error is 50.05 for n_refs = 30
The test error is 71.57 for n_refs = 30
Biased
The train error is 65.69 for n_refs = 30
The test error is 111.6 for n_refs = 30


### Bulk

#### Bulk - 0.3 SOAP 

In [335]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos3,
                                                          bulk_train_index, bulk_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos3,
                                                          bulk_biased_train_index, bulk_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 12.38 for SOAP
The test error is 18.59 for SOAP
Biased
The train error is 11.69 for SOAP
The test error is 23.14 for SOAP


#### Bulk- 0.1 SOAP 

In [336]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos1,
                                                          bulk_train_index, bulk_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(bulk_soap, bulk_dos1,
                                                          bulk_biased_train_index, bulk_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))


Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 31.49 for SOAP
The test error is 38.62 for SOAP
Biased
The train error is 30.82 for SOAP
The test error is 44.88 for SOAP


#### Bulk - 0.3 Kernel

In [337]:
regularization = 1e-1
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_200, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            regularization, bulk_kMM_200)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 200".format(loss_dos))
print ("The test error is {:.4} for n_refs = 200".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_100, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            regularization, bulk_kMM_100)
print ("Biased")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))


Smearing: 0.3
Unbiased
The train error is 10.02 for n_refs = 200
The test error is 14.86 for n_refs = 200
Biased
The train error is 13.22 for n_refs = 100
The test error is 21.07 for n_refs = 100


#### Bulk - 0.1 Kernel 

In [338]:
regularization = 1e-1
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_200, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            regularization, bulk_kMM_200)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 200".format(loss_dos))
print ("The test error is {:.4} for n_refs = 200".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(bulk_kernel_100, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            regularization, bulk_kMM_100)
print ("Biased")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 29.26 for n_refs = 200
The test error is 36.1 for n_refs = 200
Biased
The train error is 32.96 for n_refs = 100
The test error is 42.52 for n_refs = 100


### Total

#### Total - 0.3 SOAP

In [339]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.3")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos3,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.3
Regularization: 0.01
Unbiased
The train error is 18.15 for SOAP
The test error is 22.61 for SOAP
Biased
The train error is 17.23 for SOAP
The test error is 27.82 for SOAP
Holdout
The train error is 16.5 for SOAP
The test error is 31.67 for SOAP


#### Total - 0.1 SOAP

In [340]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_dos1,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 32.14 for SOAP
The test error is 37.45 for SOAP
Biased
The train error is 31.24 for SOAP
The test error is 43.32 for SOAP
Holdout
The train error is 31.64 for SOAP
The test error is 42.62 for SOAP


#### Total - 0.3 Kernel

In [341]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos3,
                                                            total_train_index, total_test_index,
                                                            regularization, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            regularization, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_100, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            regularization, total_kMM_100)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 11.61 for n_refs = 150
The test error is 16.82 for n_refs = 150
Biased
The train error is 10.85 for n_refs = 150
The test error is 21.7 for n_refs = 150
Holdout
The train error is 12.57 for n_refs = 100
The test error is 25.95 for n_refs = 100


#### Total - 0.1 Kernel

In [342]:
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-1, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-1, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, total_kMM_150)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 31.01 for n_refs = 150
The test error is 34.04 for n_refs = 150
Biased
The train error is 30.28 for n_refs = 150
The test error is 39.04 for n_refs = 150
Holdout
The train error is 26.96 for n_refs = 150
The test error is 38.11 for n_refs = 150


### Total Aligned

#### Total Aligned - 0.3 SOAP

In [343]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.3")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos3,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.3
Regularization: 0.01
Unbiased
The train error is 17.35 for SOAP
The test error is 21.77 for SOAP
Biased
The train error is 16.4 for SOAP
The test error is 27.2 for SOAP
Holdout
The train error is 16.03 for SOAP
The test error is 29.26 for SOAP


#### Total Aligned - 0.1 SOAP

In [344]:
regularization = 1e-2
U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          total_train_index, total_test_index,
                                                          regularization)
print ("Smearing: 0.1")
print ("Regularization: {}".format(regularization))
print ("Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          total_biased_train_index, total_biased_test_index,
                                                          regularization)
print ("Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

U_weight_s01, loss_dos, test_loss_dos =normal_reg_train_A(total_soap, total_aligned_dos1,
                                                          holdout_train_index, holdout_test_index,
                                                          regularization)

print ("Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Smearing: 0.1
Regularization: 0.01
Unbiased
The train error is 32.12 for SOAP
The test error is 37.33 for SOAP
Biased
The train error is 31.2 for SOAP
The test error is 43.42 for SOAP
Holdout
The train error is 31.72 for SOAP
The test error is 41.71 for SOAP


#### Total Aligned - 0.3 Kernel

In [345]:
regularization = 1e-2
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            regularization, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            regularization, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_100, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            regularization, total_kMM_100)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 100".format(loss_dos))
print ("The test error is {:.4} for n_refs = 100".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 11.37 for n_refs = 150
The test error is 16.56 for n_refs = 150
Biased
The train error is 10.61 for n_refs = 150
The test error is 21.36 for n_refs = 150
Holdout
The train error is 12.38 for n_refs = 100
The test error is 24.22 for n_refs = 100


#### Total Aligned - 0.1 Kernel

In [346]:
U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-1, total_kMM_150)
print ("Smearing: 0.3")
print ("Unbiased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-1, total_kMM_150)
print ("Biased")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

U_weight_s03, loss_dos , test_loss_dos = kernel_reg_train_A(total_kernel_150, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, total_kMM_150)
print ("Holdout")
print ("The train error is {:.4} for n_refs = 150".format(loss_dos))
print ("The test error is {:.4} for n_refs = 150".format(test_loss_dos))

Smearing: 0.3
Unbiased
The train error is 31.11 for n_refs = 150
The test error is 34.11 for n_refs = 150
Biased
The train error is 30.38 for n_refs = 150
The test error is 39.31 for n_refs = 150
Holdout
The train error is 27.18 for n_refs = 150
The test error is 37.71 for n_refs = 150


## Baseline GD model - Adam 

### Surfaces

#### Surfaces - 0.3 SOAP

In [347]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos3, surface_train_index, surface_test_index, 1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████▏                                                                                                   | 1010/10000 [00:31<03:40, 40.77it/s, lowest_mse=0.00999, pred_loss=64, trigger=0]

The batch_size is now:  32


Epoch: 2023:  20%|█████████████████████▉                                                                                       | 2017/10000 [00:48<01:52, 70.84it/s, lowest_mse=0.00724, pred_loss=54.5, trigger=0]

The batch_size is now:  64


Epoch: 3035:  30%|████████████████████████████████▉                                                                            | 3022/10000 [00:59<01:03, 109.19it/s, lowest_mse=0.0065, pred_loss=51.6, trigger=0]

...

The batch_size is now:  512


Epoch: 6041:  60%|█████████████████████████████████████████████████████████████████▏                                          | 6033/10000 [01:20<00:27, 142.71it/s, lowest_mse=0.00555, pred_loss=47.9, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [01:27<00:37, 80.02it/s, lowest_mse=0.00534, pred_loss=46.8, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 46.81 for SOAP
The test error is 65.52 for SOAP


In [348]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos3,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1013:  10%|███████████▏                                                                                                   | 1011/10000 [00:31<03:46, 39.61it/s, lowest_mse=0.012, pred_loss=73.4, trigger=0]

The batch_size is now:  32


Epoch: 2020:  20%|█████████████████████▉                                                                                       | 2015/10000 [00:50<02:04, 64.12it/s, lowest_mse=0.00899, pred_loss=63.4, trigger=0]

The batch_size is now:  64


Epoch: 3019:  30%|████████████████████████████████▉                                                                            | 3018/10000 [01:03<02:21, 49.34it/s, lowest_mse=0.00814, pred_loss=60.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6035:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6034/10000 [01:26<00:35, 112.03it/s, lowest_mse=0.00723, pred_loss=57, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [01:33<00:39, 74.82it/s, lowest_mse=0.00717, pred_loss=56.6, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 56.62 for SOAP
The test error is 100.3 for SOAP


#### Surfaces - 0.1 SOAP

In [349]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos1, 
                                                       surface_train_index, surface_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████▎                                                                                                    | 1012/10000 [00:32<03:32, 42.40it/s, lowest_mse=0.0265, pred_loss=76, trigger=0]

The batch_size is now:  32


Epoch: 2024:  20%|██████████████████████▏                                                                                       | 2020/10000 [00:49<01:49, 73.12it/s, lowest_mse=0.0219, pred_loss=69.1, trigger=0]

The batch_size is now:  64


Epoch: 3035:  30%|█████████████████████████████████▎                                                                            | 3032/10000 [01:00<01:10, 98.18it/s, lowest_mse=0.0204, pred_loss=66.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6041:  60%|█████████████████████████████████████████████████████████████████▊                                           | 6035/10000 [01:22<00:28, 140.42it/s, lowest_mse=0.0183, pred_loss=63.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:29<00:38, 78.23it/s, lowest_mse=0.0177, pred_loss=62.2, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 62.23 for SOAP
The test error is 80.45 for SOAP


In [350]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_dos1,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████                                                                                                   | 1010/10000 [00:31<03:46, 39.72it/s, lowest_mse=0.0303, pred_loss=82.3, trigger=0]

The batch_size is now:  32


Epoch: 2022:  20%|██████████████████████▏                                                                                       | 2015/10000 [00:49<02:00, 66.26it/s, lowest_mse=0.0262, pred_loss=76.5, trigger=0]

The batch_size is now:  64


Epoch: 3032:  30%|████████████████████████████████▉                                                                            | 3022/10000 [01:00<01:08, 101.27it/s, lowest_mse=0.0247, pred_loss=74.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6039:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6027/10000 [01:23<00:30, 132.29it/s, lowest_mse=0.0228, pred_loss=71.4, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:30<00:38, 77.42it/s, lowest_mse=0.0226, pred_loss=71.1, trigger=0]

The batch_size is now:  2048
Adam Holdout
The train error is 71.06 for SOAP
The test error is 112.1 for SOAP


#### Surfaces - 0.3 Kernel

In [351]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 828:   8%|█████████▎                                                                                                       | 828/10000 [00:10<01:33, 98.16it/s, lowest_mse=0.0289, pred_loss=109, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|█████████████████▉                                                                                           | 1641/10000 [00:16<00:54, 154.13it/s, lowest_mse=0.0202, pred_loss=91.1, trigger=0]

The batch_size is now:  64


Epoch: 2461:  24%|██████████████████████████▌                                                                                  | 2439/10000 [00:21<00:38, 198.88it/s, lowest_mse=0.0159, pred_loss=80.7, trigger=0]

...

The batch_size is now:  512


Epoch: 4871:  49%|████████████████████████████████████████████████████▉                                                        | 4862/10000 [00:31<00:21, 238.87it/s, lowest_mse=0.0107, pred_loss=66.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:34<00:26, 164.11it/s, lowest_mse=0.00988, pred_loss=63.7, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 63.67 for KERNEL
The test error is 69.17 for KERNEL


In [352]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 831:   8%|█████████▎                                                                                                      | 826/10000 [00:09<01:28, 103.85it/s, lowest_mse=0.0274, pred_loss=111, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|██████████████████                                                                                             | 1629/10000 [00:15<00:56, 147.49it/s, lowest_mse=0.0198, pred_loss=94, trigger=0]

The batch_size is now:  64


Epoch: 2462:  25%|██████████████████████████▊                                                                                  | 2457/10000 [00:20<00:34, 215.62it/s, lowest_mse=0.0157, pred_loss=83.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4865:  49%|████████████████████████████████████████████████████▉                                                        | 4858/10000 [00:31<00:25, 202.65it/s, lowest_mse=0.0106, pred_loss=69.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:35<00:27, 160.24it/s, lowest_mse=0.00968, pred_loss=65.8, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 65.8 for KERNEL
The test error is 101.3 for KERNEL


#### Surfaces - 0.1 Kernel

In [353]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos1,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 831:   8%|█████████▎                                                                                                      | 826/10000 [00:09<01:27, 104.82it/s, lowest_mse=0.0518, pred_loss=106, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|█████████████████▉                                                                                            | 1629/10000 [00:15<00:57, 146.78it/s, lowest_mse=0.042, pred_loss=95.8, trigger=0]

The batch_size is now:  64


Epoch: 2463:  25%|██████████████████████████▊                                                                                  | 2458/10000 [00:20<00:34, 217.25it/s, lowest_mse=0.0369, pred_loss=89.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4873:  48%|█████████████████████████████████████████████████████▊                                                         | 4850/10000 [00:30<00:22, 227.34it/s, lowest_mse=0.0293, pred_loss=80, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|██████████████████████████████████████████████████████████████▍                                                | 5629/10000 [00:33<00:26, 167.66it/s, lowest_mse=0.0278, pred_loss=78, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 77.95 for KERNEL
The test error is 83.08 for KERNEL


In [354]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-1, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 829:   8%|█████████▎                                                                                                       | 824/10000 [00:09<01:35, 95.90it/s, lowest_mse=0.0283, pred_loss=112, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|██████████████████▏                                                                                            | 1641/10000 [00:15<00:54, 154.50it/s, lowest_mse=0.0202, pred_loss=95, trigger=0]

The batch_size is now:  64


Epoch: 2460:  24%|███████████████████████████▏                                                                                   | 2445/10000 [00:20<00:37, 200.96it/s, lowest_mse=0.0162, pred_loss=85, trigger=0]

...

The batch_size is now:  512


Epoch: 4872:  49%|█████████████████████████████████████████████████████                                                        | 4869/10000 [00:30<00:21, 236.65it/s, lowest_mse=0.0108, pred_loss=69.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:33<00:26, 167.59it/s, lowest_mse=0.00985, pred_loss=66.4, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 66.37 for KERNEL
The test error is 101.5 for KERNEL


### Surfaces Aligned

#### Surfaces Aligned - 0.3 SOAP

In [355]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index, 1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████                                                                                                   | 1010/10000 [00:30<03:40, 40.74it/s, lowest_mse=0.0106, pred_loss=64.5, trigger=0]

The batch_size is now:  32


Epoch: 2024:  20%|██████████████████████▍                                                                                        | 2019/10000 [00:47<01:48, 73.36it/s, lowest_mse=0.00741, pred_loss=54, trigger=0]

The batch_size is now:  64


Epoch: 3033:  30%|████████████████████████████████▋                                                                           | 3030/10000 [00:58<01:03, 109.37it/s, lowest_mse=0.00659, pred_loss=50.9, trigger=0]

...

The batch_size is now:  512


Epoch: 6041:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6028/10000 [01:21<00:28, 141.58it/s, lowest_mse=0.00559, pred_loss=47, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▊                                 | 7015/10000 [01:28<00:37, 79.05it/s, lowest_mse=0.00538, pred_loss=46, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 45.96 for SOAP
The test error is 62.37 for SOAP


In [356]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos3,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████                                                                                                   | 1010/10000 [00:30<03:41, 40.57it/s, lowest_mse=0.0119, pred_loss=71.6, trigger=0]

The batch_size is now:  32


Epoch: 2022:  20%|█████████████████████▉                                                                                       | 2013/10000 [00:49<02:06, 63.34it/s, lowest_mse=0.00902, pred_loss=62.3, trigger=0]

The batch_size is now:  64


Epoch: 3035:  30%|████████████████████████████████▋                                                                           | 3025/10000 [01:00<01:02, 112.34it/s, lowest_mse=0.00816, pred_loss=59.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6034:  60%|█████████████████████████████████████████████████████████████████                                           | 6026/10000 [01:23<00:32, 121.86it/s, lowest_mse=0.00728, pred_loss=56.1, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [01:32<00:39, 75.97it/s, lowest_mse=0.00722, pred_loss=55.7, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 55.68 for SOAP
The test error is 97.52 for SOAP


#### Surfaces Aligned - 0.1 SOAP

In [357]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos1, 
                                                       surface_train_index, surface_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████                                                                                                   | 1009/10000 [00:30<03:47, 39.59it/s, lowest_mse=0.0266, pred_loss=75.2, trigger=0]

The batch_size is now:  32


Epoch: 2023:  20%|██████████████████████▍                                                                                        | 2023/10000 [00:49<01:46, 74.95it/s, lowest_mse=0.022, pred_loss=68.4, trigger=0]

The batch_size is now:  64


Epoch: 3035:  30%|█████████████████████████████████▋                                                                             | 3031/10000 [00:59<01:00, 114.64it/s, lowest_mse=0.0205, pred_loss=66, trigger=0]

...

The batch_size is now:  512


Epoch: 6041:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6029/10000 [01:21<00:28, 140.81it/s, lowest_mse=0.0182, pred_loss=62.3, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:28<00:37, 79.51it/s, lowest_mse=0.0177, pred_loss=61.5, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 61.45 for SOAP
The test error is 77.85 for SOAP


In [358]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(surface_soap, surface_aligned_dos1,
                                                       surface_holdout_train_index, surface_holdout_test_index, 1e-1,
                                                       10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1014:  10%|███████████                                                                                                   | 1010/10000 [00:30<03:37, 41.29it/s, lowest_mse=0.0302, pred_loss=81.3, trigger=0]

The batch_size is now:  32


Epoch: 2024:  20%|██████████████████████▎                                                                                       | 2024/10000 [00:47<01:43, 76.79it/s, lowest_mse=0.0261, pred_loss=75.7, trigger=0]

The batch_size is now:  64


Epoch: 3035:  30%|█████████████████████████████████                                                                            | 3032/10000 [00:57<01:00, 115.80it/s, lowest_mse=0.0247, pred_loss=73.5, trigger=0]

...

The batch_size is now:  512


Epoch: 6041:  60%|█████████████████████████████████████████████████████████████████▊                                           | 6033/10000 [01:19<00:27, 141.80it/s, lowest_mse=0.0228, pred_loss=70.7, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [01:26<00:36, 81.11it/s, lowest_mse=0.0226, pred_loss=70.4, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 70.38 for SOAP
The test error is 109.9 for SOAP


#### Surfaces Aligned - 0.3 Kernel

In [359]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 830:   8%|█████████▏                                                                                                      | 825/10000 [00:09<01:29, 102.77it/s, lowest_mse=0.0297, pred_loss=108, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|█████████████████▊                                                                                           | 1634/10000 [00:15<00:56, 149.15it/s, lowest_mse=0.0204, pred_loss=89.5, trigger=0]

The batch_size is now:  64


Epoch: 2460:  25%|██████████████████████████▊                                                                                  | 2458/10000 [00:20<00:36, 206.68it/s, lowest_mse=0.0159, pred_loss=79.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4872:  49%|████████████████████████████████████████████████████▉                                                        | 4858/10000 [00:30<00:21, 237.16it/s, lowest_mse=0.0106, pred_loss=64.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:33<00:26, 166.98it/s, lowest_mse=0.00982, pred_loss=62.1, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 62.07 for KERNEL
The test error is 67.28 for KERNEL


In [360]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 831:   8%|█████████▍                                                                                                        | 825/10000 [00:09<01:29, 103.09it/s, lowest_mse=0.03, pred_loss=114, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|██████████████████▏                                                                                            | 1644/10000 [00:15<00:54, 153.81it/s, lowest_mse=0.0214, pred_loss=96, trigger=0]

The batch_size is now:  64


Epoch: 2459:  24%|██████████████████████████▌                                                                                  | 2439/10000 [00:20<00:38, 197.21it/s, lowest_mse=0.0169, pred_loss=85.2, trigger=0]

...

The batch_size is now:  512


Epoch: 4871:  49%|█████████████████████████████████████████████████████                                                        | 4863/10000 [00:30<00:22, 233.08it/s, lowest_mse=0.0108, pred_loss=68.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:33<00:26, 165.88it/s, lowest_mse=0.00982, pred_loss=64.9, trigger=0]

The batch_size is now:  2048
Adam Holdout
The train error is 64.93 for KERNEL
The test error is 101.0 for KERNEL


#### Surfaces Aligned - 0.1 Kernel

In [361]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos1,
                                                              surface_train_index, surface_test_index,
                                                              surface_kMM_30, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 831:   8%|█████████▎                                                                                                      | 826/10000 [00:09<01:28, 103.87it/s, lowest_mse=0.0525, pred_loss=106, trigger=0]

The batch_size is now:  32


Epoch: 1645:  16%|█████████████████▉                                                                                           | 1640/10000 [00:15<00:54, 152.16it/s, lowest_mse=0.0421, pred_loss=94.6, trigger=0]

The batch_size is now:  64


Epoch: 2460:  24%|██████████████████████████▉                                                                                   | 2444/10000 [00:20<00:37, 200.65it/s, lowest_mse=0.037, pred_loss=88.7, trigger=0]

...

The batch_size is now:  512


Epoch: 4872:  49%|█████████████████████████████████████████████████████                                                        | 4868/10000 [00:30<00:21, 236.19it/s, lowest_mse=0.0292, pred_loss=78.8, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▎                                               | 5629/10000 [00:33<00:26, 167.14it/s, lowest_mse=0.0277, pred_loss=76.8, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 76.81 for KERNEL
The test error is 81.94 for KERNEL


In [362]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(surface_kernel_30, surface_aligned_dos3,
                                                              surface_holdout_train_index, surface_holdout_test_index,
                                                              surface_kMM_30, 1e-1, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 831:   8%|█████████▎                                                                                                      | 826/10000 [00:09<01:28, 104.00it/s, lowest_mse=0.0272, pred_loss=108, trigger=0]

The batch_size is now:  32


Epoch: 1638:  16%|█████████████████▋                                                                                           | 1625/10000 [00:16<01:10, 119.29it/s, lowest_mse=0.0198, pred_loss=92.2, trigger=0]

The batch_size is now:  64


Epoch: 2452:  24%|██████████████████████████▌                                                                                  | 2433/10000 [00:22<00:45, 165.32it/s, lowest_mse=0.0158, pred_loss=82.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4867:  48%|████████████████████████████████████████████████████▊                                                        | 4850/10000 [00:33<00:23, 218.62it/s, lowest_mse=0.0107, pred_loss=67.8, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|████████████████████████████████████████████████████████████▊                                               | 5629/10000 [00:37<00:28, 151.35it/s, lowest_mse=0.00978, pred_loss=64.8, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 64.83 for KERNEL
The test error is 99.49 for KERNEL


### Bulk

#### Bulk - 0.3 SOAP

In [363]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos3,
                                                       bulk_train_index, bulk_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:22<15:48,  9.48it/s, lowest_mse=0.00442, pred_loss=21.9, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|█████████████████████▉                                                                                       | 2008/10000 [03:36<07:58, 16.70it/s, lowest_mse=0.00367, pred_loss=18.9, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3014/10000 [04:19<03:55, 29.68it/s, lowest_mse=0.00332, pred_loss=18.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6030:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6023/10000 [05:16<00:52, 76.47it/s, lowest_mse=0.00291, pred_loss=16.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [05:27<02:19, 21.45it/s, lowest_mse=0.00283, pred_loss=15.9, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 15.94 for SOAP
The test error is 17.75 for SOAP


In [364]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos3,
                                                       bulk_biased_train_index, bulk_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1004:  10%|██████████▉                                                                                                  | 1004/10000 [02:55<25:51,  5.80it/s, lowest_mse=0.00446, pred_loss=20.7, trigger=0]

The batch_size is now:  32


Epoch: 2007:  20%|█████████████████████▊                                                                                       | 2006/10000 [04:31<19:17,  6.91it/s, lowest_mse=0.00367, pred_loss=18.4, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3011/10000 [05:18<04:27, 26.17it/s, lowest_mse=0.00331, pred_loss=17.1, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6023/10000 [06:15<00:49, 80.28it/s, lowest_mse=0.00289, pred_loss=15.4, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:25<02:44, 18.19it/s, lowest_mse=0.00281, pred_loss=15.2, trigger=0]

The batch_size is now:  2048
Adam Holdout
The train error is 15.23 for SOAP
The test error is 23.09 for SOAP


#### Bulk - 0.1 Soap

In [365]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos1,
                                                       bulk_train_index, bulk_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:15<15:46,  9.50it/s, lowest_mse=0.0238, pred_loss=40.4, trigger=0]

The batch_size is now:  32


Epoch: 2010:  20%|██████████████████████▎                                                                                        | 2008/10000 [03:27<07:52, 16.91it/s, lowest_mse=0.022, pred_loss=38.3, trigger=0]

The batch_size is now:  64


Epoch: 3015:  30%|█████████████████████████████████▏                                                                            | 3012/10000 [04:09<03:54, 29.77it/s, lowest_mse=0.0211, pred_loss=37.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6034:  60%|████████████████████████████████████████████████████████████████████▋                                             | 6024/10000 [05:03<00:46, 86.42it/s, lowest_mse=0.02, pred_loss=36, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [05:13<02:13, 22.41it/s, lowest_mse=0.0197, pred_loss=35.8, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 35.76 for SOAP
The test error is 38.11 for SOAP


In [366]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(bulk_soap, bulk_dos1,
                                                       bulk_biased_train_index, bulk_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████▏                                                                                                   | 1005/10000 [02:21<15:59,  9.37it/s, lowest_mse=0.025, pred_loss=39.4, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████▎                                                                                        | 2008/10000 [03:36<08:04, 16.50it/s, lowest_mse=0.023, pred_loss=37.6, trigger=0]

The batch_size is now:  64


Epoch: 3015:  30%|█████████████████████████████████▏                                                                            | 3012/10000 [04:17<03:49, 30.47it/s, lowest_mse=0.0221, pred_loss=36.6, trigger=0]

...

The batch_size is now:  512


Epoch: 6033:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6024/10000 [05:09<00:47, 84.15it/s, lowest_mse=0.0209, pred_loss=35.3, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [05:18<02:15, 22.03it/s, lowest_mse=0.0206, pred_loss=35.1, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 35.11 for SOAP
The test error is 44.07 for SOAP


#### Bulk - 0.3 Kernel

In [367]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_200, bulk_dos3,
                                                              bulk_train_index, bulk_test_index,
                                                              bulk_kMM_200, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:16<11:52, 12.91it/s, lowest_mse=0.0203, pred_loss=69.8, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▋                                                                                            | 1613/10000 [01:57<05:40, 24.65it/s, lowest_mse=0.0152, pred_loss=47.1, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▌                                                                                   | 2419/10000 [02:20<03:02, 41.53it/s, lowest_mse=0.0137, pred_loss=41.1, trigger=0]

...

The batch_size is now:  512


Epoch: 4850:  48%|█████████████████████████████████████████████████████▎                                                        | 4849/10000 [02:53<00:46, 111.40it/s, lowest_mse=0.013, pred_loss=41.2, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:59<02:19, 31.33it/s, lowest_mse=0.0128, pred_loss=33.9, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 33.93 for KERNEL
The test error is 35.07 for KERNEL


In [368]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_100, bulk_dos3,
                                                              bulk_biased_train_index, bulk_biased_test_index,
                                                              bulk_kMM_100, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████▏                                                                                                        | 809/10000 [01:00<08:57, 17.09it/s, lowest_mse=0.0216, pred_loss=50, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▋                                                                                            | 1613/10000 [01:35<05:32, 25.22it/s, lowest_mse=0.0161, pred_loss=40.5, trigger=0]

The batch_size is now:  64


Epoch: 2425:  24%|██████████████████████████▌                                                                                   | 2420/10000 [01:56<02:41, 46.87it/s, lowest_mse=0.0142, pred_loss=37.4, trigger=0]

...

The batch_size is now:  512


Epoch: 4853:  48%|████████████████████████████████████████████████████▊                                                        | 4843/10000 [02:25<00:42, 122.31it/s, lowest_mse=0.0128, pred_loss=33.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:30<01:56, 37.47it/s, lowest_mse=0.0126, pred_loss=32.2, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 32.2 for KERNEL
The test error is 45.04 for KERNEL


#### Bulk - 0.1 Kernel

In [369]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_200, bulk_dos1,
                                                              bulk_train_index, bulk_test_index,
                                                              bulk_kMM_200, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:16<11:40, 13.12it/s, lowest_mse=0.0506, pred_loss=72.5, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1614/10000 [01:56<05:36, 24.95it/s, lowest_mse=0.0434, pred_loss=59.1, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|███████████████████████████                                                                                     | 2420/10000 [02:19<02:58, 42.36it/s, lowest_mse=0.0408, pred_loss=56, trigger=0]

...

The batch_size is now:  512


Epoch: 4852:  48%|████████████████████████████████████████████████████▊                                                        | 4847/10000 [02:51<00:44, 115.91it/s, lowest_mse=0.0391, pred_loss=52.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:57<02:17, 31.78it/s, lowest_mse=0.0388, pred_loss=50.1, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 50.13 for KERNEL
The test error is 51.49 for KERNEL


In [370]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(bulk_kernel_100, bulk_dos1,
                                                              bulk_biased_train_index, bulk_biased_test_index,
                                                              bulk_kMM_100, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████▏                                                                                                        | 809/10000 [01:00<09:00, 17.00it/s, lowest_mse=0.0529, pred_loss=62, trigger=0]

The batch_size is now:  32


Epoch: 1617:  16%|██████████████████                                                                                              | 1615/10000 [01:34<04:34, 30.53it/s, lowest_mse=0.0459, pred_loss=57, trigger=0]

The batch_size is now:  64


Epoch: 2426:  24%|██████████████████████████▋                                                                                   | 2423/10000 [01:52<02:24, 52.33it/s, lowest_mse=0.0432, pred_loss=52.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4854:  48%|████████████████████████████████████████████████████▊                                                        | 4848/10000 [02:18<00:38, 134.69it/s, lowest_mse=0.0407, pred_loss=49.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:24<01:51, 39.04it/s, lowest_mse=0.0403, pred_loss=49.1, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 49.06 for KERNEL
The test error is 61.5 for KERNEL


### Total

#### Total - 0.3 Soap


In [371]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████▏                                                                                                   | 1005/10000 [02:45<18:54,  7.93it/s, lowest_mse=0.00955, pred_loss=27, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:12<09:32, 13.97it/s, lowest_mse=0.0085, pred_loss=25.6, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3013/10000 [05:01<04:26, 26.23it/s, lowest_mse=0.00794, pred_loss=24.1, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6023/10000 [06:00<00:51, 77.79it/s, lowest_mse=0.0072, pred_loss=22.8, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:11<02:37, 18.90it/s, lowest_mse=0.00707, pred_loss=22.4, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 22.44 for SOAP
The test error is 24.85 for SOAP


In [372]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:44<18:59,  7.90it/s, lowest_mse=0.00976, pred_loss=27.9, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|█████████████████████▉                                                                                       | 2009/10000 [04:14<08:57, 14.86it/s, lowest_mse=0.00866, pred_loss=24.1, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3012/10000 [05:03<04:37, 25.17it/s, lowest_mse=0.00808, pred_loss=23.5, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|██████████████████████████████████████████████████████████████████▉                                            | 6027/10000 [06:01<00:49, 79.84it/s, lowest_mse=0.00731, pred_loss=22, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:12<02:38, 18.85it/s, lowest_mse=0.00718, pred_loss=21.6, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 21.65 for SOAP
The test error is 30.46 for SOAP


In [373]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos3,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:44<19:12,  7.81it/s, lowest_mse=0.00787, pred_loss=24.5, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|█████████████████████▉                                                                                       | 2008/10000 [04:10<09:18, 14.31it/s, lowest_mse=0.00689, pred_loss=22.8, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3012/10000 [04:59<04:36, 25.30it/s, lowest_mse=0.00639, pred_loss=21.8, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6030/10000 [06:00<00:48, 82.04it/s, lowest_mse=0.00574, pred_loss=20.6, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:10<02:37, 18.92it/s, lowest_mse=0.00563, pred_loss=20.2, trigger=0]


The batch_size is now:  2048
Adam Holdout
The train error is 20.22 for SOAP
The test error is 34.56 for SOAP


#### Total - 0.1 Soap

In [374]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:41<18:52,  7.95it/s, lowest_mse=0.0296, pred_loss=41.1, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:07<09:33, 13.93it/s, lowest_mse=0.0276, pred_loss=39.9, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████                                                                             | 3011/10000 [04:55<04:40, 24.87it/s, lowest_mse=0.0266, pred_loss=38.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6024/10000 [05:59<00:53, 73.88it/s, lowest_mse=0.0251, pred_loss=37.1, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [06:11<02:37, 18.90it/s, lowest_mse=0.0249, pred_loss=36.8, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 36.82 for SOAP
The test error is 38.99 for SOAP


In [375]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:42<18:50,  7.96it/s, lowest_mse=0.0309, pred_loss=40.2, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:08<09:27, 14.09it/s, lowest_mse=0.0287, pred_loss=38.4, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████                                                                             | 3011/10000 [04:56<04:39, 24.97it/s, lowest_mse=0.0276, pred_loss=37.5, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6025/10000 [05:54<00:48, 81.36it/s, lowest_mse=0.0261, pred_loss=36.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|██████████████████████████████████████████████████████████████████████████████▌                                 | 7015/10000 [06:06<02:35, 19.14it/s, lowest_mse=0.0258, pred_loss=36, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 36.01 for SOAP
The test error is 44.75 for SOAP


In [376]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_dos1,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████▏                                                                                                   | 1005/10000 [02:55<19:06,  7.84it/s, lowest_mse=0.028, pred_loss=40.3, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████▎                                                                                        | 2009/10000 [04:22<09:07, 14.59it/s, lowest_mse=0.026, pred_loss=38.5, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████▍                                                                             | 3012/10000 [05:11<04:35, 25.32it/s, lowest_mse=0.025, pred_loss=37.4, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6026/10000 [06:10<00:49, 80.33it/s, lowest_mse=0.0237, pred_loss=36.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [06:20<02:42, 18.43it/s, lowest_mse=0.0234, pred_loss=35.9, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 35.93 for SOAP
The test error is 44.63 for SOAP


#### Total - 0.3 Kernel

In [377]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos3,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:17<11:57, 12.81it/s, lowest_mse=0.0291, pred_loss=62.9, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|██████████████████                                                                                              | 1615/10000 [01:59<05:25, 25.73it/s, lowest_mse=0.0235, pred_loss=48, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▌                                                                                   | 2420/10000 [02:23<03:11, 39.62it/s, lowest_mse=0.0217, pred_loss=41.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4849:  48%|████████████████████████████████████████████████████▊                                                        | 4846/10000 [02:57<00:48, 107.35it/s, lowest_mse=0.0203, pred_loss=50.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:03<02:22, 30.60it/s, lowest_mse=0.0202, pred_loss=37.9, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 37.9 for KERNEL
The test error is 39.15 for KERNEL


In [378]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos3,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 807/10000 [01:18<13:19, 11.50it/s, lowest_mse=0.0296, pred_loss=61.9, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▋                                                                                            | 1613/10000 [02:01<05:51, 23.88it/s, lowest_mse=0.0238, pred_loss=48.9, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▉                                                                                    | 2423/10000 [02:25<02:59, 42.11it/s, lowest_mse=0.022, pred_loss=41.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4851:  48%|████████████████████████████████████████████████████▊                                                        | 4850/10000 [02:58<00:43, 117.32it/s, lowest_mse=0.0207, pred_loss=47.3, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:04<02:23, 30.50it/s, lowest_mse=0.0206, pred_loss=36.6, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 36.64 for KERNEL
The test error is 46.88 for KERNEL


In [379]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_dos3,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████                                                                                                       | 809/10000 [01:13<10:55, 14.02it/s, lowest_mse=0.0285, pred_loss=50.3, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1614/10000 [01:53<05:35, 24.97it/s, lowest_mse=0.0226, pred_loss=43.7, trigger=0]

The batch_size is now:  64


Epoch: 2425:  24%|███████████████████████████                                                                                     | 2421/10000 [02:15<02:48, 44.97it/s, lowest_mse=0.0205, pred_loss=41, trigger=0]

...

The batch_size is now:  512


Epoch: 4852:  48%|████████████████████████████████████████████████████▊                                                        | 4850/10000 [02:44<00:40, 125.66it/s, lowest_mse=0.0188, pred_loss=39.8, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:49<02:11, 33.25it/s, lowest_mse=0.0185, pred_loss=36.7, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 36.69 for KERNEL
The test error is 45.29 for KERNEL


#### Total - 0.1 Kernel 

In [380]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos1,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:17<11:55, 12.85it/s, lowest_mse=0.0581, pred_loss=67.5, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [02:00<05:33, 25.14it/s, lowest_mse=0.0516, pred_loss=59.1, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:24<03:07, 40.34it/s, lowest_mse=0.0491, pred_loss=55.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4851:  48%|████████████████████████████████████████████████████▋                                                        | 4838/10000 [02:57<00:46, 110.47it/s, lowest_mse=0.0468, pred_loss=55.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:03<02:22, 30.65it/s, lowest_mse=0.0464, pred_loss=50.3, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 50.32 for KERNEL
The test error is 51.39 for KERNEL


In [381]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_dos1,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:18<11:56, 12.83it/s, lowest_mse=0.0612, pred_loss=70.6, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [02:01<05:26, 25.69it/s, lowest_mse=0.0539, pred_loss=56.7, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:24<02:56, 42.90it/s, lowest_mse=0.0511, pred_loss=54.2, trigger=0]

...

The batch_size is now:  512


Epoch: 4852:  48%|████████████████████████████████████████████████████▋                                                        | 4839/10000 [02:56<00:45, 113.02it/s, lowest_mse=0.0487, pred_loss=54.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:01<02:21, 30.95it/s, lowest_mse=0.0483, pred_loss=49.2, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 49.23 for KERNEL
The test error is 58.51 for KERNEL


In [382]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_dos1,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████                                                                                                       | 809/10000 [01:14<10:56, 14.00it/s, lowest_mse=0.0588, pred_loss=59.5, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1615/10000 [01:54<05:24, 25.80it/s, lowest_mse=0.0518, pred_loss=56.9, trigger=0]

The batch_size is now:  64


Epoch: 2425:  24%|██████████████████████████▋                                                                                   | 2423/10000 [02:16<02:41, 46.89it/s, lowest_mse=0.0489, pred_loss=54.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4855:  49%|████████████████████████████████████████████████████▉                                                        | 4851/10000 [02:46<00:39, 128.99it/s, lowest_mse=0.0461, pred_loss=51.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:51<02:13, 32.84it/s, lowest_mse=0.0457, pred_loss=50.2, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 50.2 for KERNEL
The test error is 55.09 for KERNEL


### Total Aligned

#### Total Aligned - 0.3 Soap


In [383]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:45<19:02,  7.87it/s, lowest_mse=0.00853, pred_loss=26.2, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|█████████████████████▉                                                                                       | 2009/10000 [04:13<08:54, 14.95it/s, lowest_mse=0.00752, pred_loss=24.1, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████▍                                                                             | 3011/10000 [05:01<04:49, 24.15it/s, lowest_mse=0.007, pred_loss=23.1, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6031/10000 [06:03<00:47, 83.35it/s, lowest_mse=0.00631, pred_loss=21.7, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:13<02:39, 18.76it/s, lowest_mse=0.00619, pred_loss=21.4, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 21.37 for SOAP
The test error is 23.7 for SOAP


In [384]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:43<19:08,  7.83it/s, lowest_mse=0.00865, pred_loss=25.3, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:11<09:28, 14.07it/s, lowest_mse=0.0076, pred_loss=23.3, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3011/10000 [05:00<04:39, 25.01it/s, lowest_mse=0.00706, pred_loss=22.1, trigger=0]

...

The batch_size is now:  512


Epoch: 6029:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6026/10000 [06:00<00:57, 69.16it/s, lowest_mse=0.00636, pred_loss=20.9, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:12<02:38, 18.82it/s, lowest_mse=0.00624, pred_loss=20.5, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 20.51 for SOAP
The test error is 29.5 for SOAP


In [385]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos3,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|██████████▉                                                                                                  | 1005/10000 [02:46<19:17,  7.77it/s, lowest_mse=0.00729, pred_loss=24.6, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|█████████████████████▉                                                                                       | 2008/10000 [04:13<09:27, 14.09it/s, lowest_mse=0.00635, pred_loss=22.5, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|████████████████████████████████▊                                                                            | 3011/10000 [05:02<04:41, 24.86it/s, lowest_mse=0.00587, pred_loss=21.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|█████████████████████████████████████████████████████████████████▋                                           | 6029/10000 [06:07<00:50, 79.35it/s, lowest_mse=0.00525, pred_loss=19.9, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|████████████████████████████████████████████████████████████████████████████▍                                | 7015/10000 [06:19<02:41, 18.49it/s, lowest_mse=0.00515, pred_loss=19.6, trigger=0]

The batch_size is now:  2048
Adam Holdout
The train error is 19.6 for SOAP
The test error is 31.79 for SOAP


#### Total Aligned - 0.1 Soap

In [386]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       total_train_index, total_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:44<19:02,  7.88it/s, lowest_mse=0.0286, pred_loss=40.7, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:10<09:27, 14.09it/s, lowest_mse=0.0266, pred_loss=39.2, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████                                                                             | 3011/10000 [04:58<04:45, 24.46it/s, lowest_mse=0.0256, pred_loss=38.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6032:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6023/10000 [05:59<00:50, 79.39it/s, lowest_mse=0.0242, pred_loss=36.9, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▊                                 | 7015/10000 [06:10<02:37, 18.92it/s, lowest_mse=0.024, pred_loss=36.6, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 36.64 for SOAP
The test error is 38.75 for SOAP


In [387]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       total_biased_train_index, total_biased_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:44<19:07,  7.84it/s, lowest_mse=0.0297, pred_loss=39.9, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:10<09:23, 14.18it/s, lowest_mse=0.0276, pred_loss=38.2, trigger=0]

The batch_size is now:  64


Epoch: 3013:  30%|█████████████████████████████████                                                                             | 3011/10000 [04:59<04:40, 24.95it/s, lowest_mse=0.0266, pred_loss=37.2, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|███████████████████████████████████████████████████████████████████▍                                            | 6024/10000 [06:00<00:51, 77.02it/s, lowest_mse=0.0251, pred_loss=36, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [06:11<02:37, 18.90it/s, lowest_mse=0.0249, pred_loss=35.8, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 35.78 for SOAP
The test error is 44.7 for SOAP


In [388]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(total_soap, total_aligned_dos1,
                                                       holdout_train_index, holdout_test_index,
                                                       1e-2, 10000, 16, 1e-3)
print ("Adam holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 1005:  10%|███████████                                                                                                   | 1005/10000 [02:46<19:10,  7.82it/s, lowest_mse=0.0274, pred_loss=40.3, trigger=0]

The batch_size is now:  32


Epoch: 2009:  20%|██████████████████████                                                                                        | 2008/10000 [04:14<09:18, 14.31it/s, lowest_mse=0.0255, pred_loss=38.3, trigger=0]

The batch_size is now:  64


Epoch: 3014:  30%|█████████████████████████████████                                                                             | 3011/10000 [05:05<04:39, 25.00it/s, lowest_mse=0.0245, pred_loss=37.3, trigger=0]

...

The batch_size is now:  512


Epoch: 6031:  60%|██████████████████████████████████████████████████████████████████▎                                           | 6029/10000 [06:07<00:53, 74.07it/s, lowest_mse=0.0232, pred_loss=36.2, trigger=0]

The batch_size is now:  1024


Epoch: 7015:  70%|█████████████████████████████████████████████████████████████████████████████▏                                | 7015/10000 [06:20<02:41, 18.45it/s, lowest_mse=0.0229, pred_loss=35.9, trigger=0]

The batch_size is now:  2048
Adam holdout
The train error is 35.94 for SOAP
The test error is 43.34 for SOAP


#### Total Aligned - 0.3 Kernel

In [389]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos3,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████▏                                                                                                       | 808/10000 [01:17<11:57, 12.82it/s, lowest_mse=0.027, pred_loss=57.9, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [01:59<05:31, 25.28it/s, lowest_mse=0.0216, pred_loss=56.9, trigger=0]

The batch_size is now:  64


Epoch: 2423:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:23<03:01, 41.82it/s, lowest_mse=0.0199, pred_loss=41.9, trigger=0]

...

The batch_size is now:  512


Epoch: 4852:  48%|████████████████████████████████████████████████████▊                                                        | 4843/10000 [02:56<00:44, 116.44it/s, lowest_mse=0.0186, pred_loss=47.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:02<02:21, 30.85it/s, lowest_mse=0.0185, pred_loss=36.9, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 36.89 for KERNEL
The test error is 38.14 for KERNEL


In [390]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos3,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-2, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:19<11:51, 12.92it/s, lowest_mse=0.0279, pred_loss=78.9, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|█████████████████▊                                                                                            | 1615/10000 [02:02<05:23, 25.92it/s, lowest_mse=0.0221, pred_loss=45.2, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:26<02:56, 43.04it/s, lowest_mse=0.0203, pred_loss=39.4, trigger=0]

...

The batch_size is now:  512


Epoch: 4846:  48%|█████████████████████████████████████████████████████▋                                                         | 4838/10000 [03:01<00:56, 91.10it/s, lowest_mse=0.019, pred_loss=54.4, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:08<02:26, 29.92it/s, lowest_mse=0.0188, pred_loss=35.7, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 35.66 for KERNEL
The test error is 46.75 for KERNEL


In [391]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_aligned_dos3,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 809:   8%|█████████                                                                                                       | 809/10000 [01:12<10:46, 14.22it/s, lowest_mse=0.0266, pred_loss=48.3, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▉                                                                                             | 1614/10000 [01:51<05:30, 25.34it/s, lowest_mse=0.021, pred_loss=42.3, trigger=0]

The batch_size is now:  64


Epoch: 2425:  24%|██████████████████████████▋                                                                                   | 2421/10000 [02:14<02:47, 45.14it/s, lowest_mse=0.0191, pred_loss=39.6, trigger=0]

...

The batch_size is now:  512


Epoch: 4853:  48%|████████████████████████████████████████████████████▊                                                        | 4850/10000 [02:42<00:40, 127.21it/s, lowest_mse=0.0175, pred_loss=39.1, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:47<02:10, 33.52it/s, lowest_mse=0.0173, pred_loss=35.9, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 35.92 for KERNEL
The test error is 43.15 for KERNEL


#### Total Aligned - 0.1 Kernel 

In [392]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos1,
                                                              total_train_index, total_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:16<11:50, 12.94it/s, lowest_mse=0.0568, pred_loss=72.7, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|██████████████████                                                                                              | 1615/10000 [02:02<06:06, 22.87it/s, lowest_mse=0.05, pred_loss=59.2, trigger=0]

The batch_size is now:  64


Epoch: 2422:  24%|██████████████████████████▌                                                                                   | 2420/10000 [02:29<03:38, 34.71it/s, lowest_mse=0.0475, pred_loss=56.7, trigger=0]

...

The batch_size is now:  512


Epoch: 4849:  48%|████████████████████████████████████████████████████▋                                                        | 4839/10000 [03:04<00:48, 105.52it/s, lowest_mse=0.0451, pred_loss=56.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:11<02:28, 29.44it/s, lowest_mse=0.0448, pred_loss=50.1, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 50.11 for KERNEL
The test error is 51.27 for KERNEL


In [393]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_150, total_aligned_dos1,
                                                              total_biased_train_index, total_biased_test_index,
                                                              total_kMM_150, 1e-1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████                                                                                                       | 808/10000 [01:22<11:49, 12.96it/s, lowest_mse=0.0585, pred_loss=68.5, trigger=0]

The batch_size is now:  32


Epoch: 1615:  16%|██████████████████                                                                                              | 1615/10000 [02:05<05:25, 25.79it/s, lowest_mse=0.0516, pred_loss=56, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|███████████████████████████▎                                                                                     | 2422/10000 [02:28<02:54, 43.35it/s, lowest_mse=0.049, pred_loss=54, trigger=0]

...

The batch_size is now:  512


Epoch: 4850:  48%|████████████████████████████████████████████████████▊                                                        | 4841/10000 [03:00<00:47, 107.61it/s, lowest_mse=0.0467, pred_loss=55.6, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [03:06<02:24, 30.17it/s, lowest_mse=0.0464, pred_loss=48.9, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 48.88 for KERNEL
The test error is 58.9 for KERNEL


In [394]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(total_kernel_100, total_aligned_dos1,
                                                              holdout_train_index, holdout_test_index,
                                                              total_kMM_100, 1e-2, 10000, 16, 1e-3)

print ("Adam holdout")
print ("The train error is {:.4} for KERNEL".format(loss_dos))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos))

Epoch: 808:   8%|█████████▏                                                                                                       | 808/10000 [01:14<11:29, 13.32it/s, lowest_mse=0.057, pred_loss=61.8, trigger=0]

The batch_size is now:  32


Epoch: 1616:  16%|█████████████████▊                                                                                            | 1615/10000 [01:54<05:20, 26.18it/s, lowest_mse=0.0502, pred_loss=55.8, trigger=0]

The batch_size is now:  64


Epoch: 2424:  24%|██████████████████████████▋                                                                                   | 2422/10000 [02:17<02:56, 43.04it/s, lowest_mse=0.0475, pred_loss=53.2, trigger=0]

...

The batch_size is now:  512


Epoch: 4853:  48%|████████████████████████████████████████████████████▊                                                        | 4843/10000 [02:47<00:41, 123.52it/s, lowest_mse=0.0449, pred_loss=51.5, trigger=0]

The batch_size is now:  1024


Epoch: 5629:  56%|█████████████████████████████████████████████████████████████▉                                                | 5629/10000 [02:52<02:13, 32.70it/s, lowest_mse=0.0445, pred_loss=50.1, trigger=0]


The batch_size is now:  2048
Adam holdout
The train error is 50.05 for KERNEL
The test error is 54.2 for KERNEL


## Baseline GD Model - LBFGS

### Surfaces

#### Surfaces - 0.3 SOAP 

In [395]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos3, surface_train_index, surface_test_index,
                                                            1e-2, 6, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.27it/s, lowest_mse=0.00371, pred_loss=39, trigger=0]

LBFGS Unbiased
The train error is 38.41 for SOAP
The test error is 61.11 for SOAP


In [396]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.61it/s, lowest_mse=0.00708, pred_loss=56.3, trigger=0]

LBFGS holdout
The train error is 56.29 for SOAP
The test error is 100.2 for SOAP


#### Surfaces - 0.1 SOAP 

In [397]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos1,
                                                            surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.35it/s, lowest_mse=0.0131, pred_loss=53.6, trigger=0]

LBFGS Unbiased
The train error is 53.56 for SOAP
The test error is 77.05 for SOAP


In [398]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.41it/s, lowest_mse=0.0222, pred_loss=70.7, trigger=0]

LBFGS holdout
The train error is 70.42 for SOAP
The test error is 112.7 for SOAP


#### Surfaces - 0.3 Kernel

In [399]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos3,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.39it/s, lowest_mse=0.00579, pred_loss=48.7, trigger=0]

LBFGS Unbiased
The train error is 48.75 for Kernel
The test error is 65.47 for Kernel


In [400]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos3,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.70it/s, lowest_mse=0.00549, pred_loss=49.5, trigger=0]

LBFGS Holdout
The train error is 49.54 for Kernel
The test error is 104.5 for Kernel


#### Surfaces - 0.1 Kernel

In [401]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos1,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.43it/s, lowest_mse=0.0165, pred_loss=60, trigger=0]

LBFGS Unbiased
The train error is 60.02 for Kernel
The test error is 76.04 for Kernel


In [402]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_dos1,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.95it/s, lowest_mse=0.0178, pred_loss=63, trigger=0]

LBFGS Holdout
The train error is 63.02 for Kernel
The test error is 120.5 for Kernel


### Surfaces Aligned

#### Surfaces Aligned - 0.3 SOAP 

In [403]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos3, surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.05it/s, lowest_mse=0.00407, pred_loss=40, trigger=0]

LBFGS Unbiased
The train error is 39.99 for SOAP
The test error is 57.27 for SOAP


In [404]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos3, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.09it/s, lowest_mse=0.00712, pred_loss=55.4, trigger=0]

LBFGS holdout
The train error is 55.32 for SOAP
The test error is 97.64 for SOAP


#### Surfaces Aligned - 0.1 SOAP 

In [405]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos1,
                                                            surface_train_index, surface_test_index,
                                                            1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:04<00:00,  2.32it/s, lowest_mse=0.0132, pred_loss=53, trigger=0]

LBFGS Unbiased
The train error is 52.98 for SOAP
The test error is 74.83 for SOAP


In [406]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(surface_soap, surface_aligned_dos1, surface_holdout_train_index, surface_holdout_test_index,
                                                            1e-1, 10, 1)
print ("LBFGS holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.28it/s, lowest_mse=0.0223, pred_loss=70, trigger=0]

LBFGS holdout
The train error is 69.98 for SOAP
The test error is 109.9 for SOAP


#### Surfaces Aligned - 0.3 Kernel

In [407]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos3,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.73it/s, lowest_mse=0.00586, pred_loss=48, trigger=0]

LBFGS Unbiased
The train error is 47.95 for Kernel
The test error is 62.27 for Kernel


In [408]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos3,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.13it/s, lowest_mse=0.0056, pred_loss=49.1, trigger=0]

LBFGS Holdout
The train error is 49.06 for Kernel
The test error is 100.9 for Kernel


#### Surfaces Aligned - 0.1 Kernel

In [409]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos1,
                                                            surface_train_index, surface_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.29it/s, lowest_mse=0.0204, pred_loss=66, trigger=0]

LBFGS Unbiased
The train error is 65.95 for Kernel
The test error is 78.35 for Kernel


In [410]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(surface_kernel_30, surface_aligned_dos1,
                                                            surface_holdout_train_index, surface_holdout_test_index,
                                                            surface_kMM_30, 1e-2, 10, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.51it/s, lowest_mse=0.0191, pred_loss=64.7, trigger=0]

LBFGS Holdout
The train error is 64.68 for Kernel
The test error is 116.1 for Kernel


### Bulk

#### Bulk - 0.3 SOAP

In [411]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:14<00:00,  1.01it/s, lowest_mse=0.00197, pred_loss=13.3, trigger=0]

LBFGS Unbiased
The train error is 13.03 for SOAP
The test error is 18.6 for SOAP


In [412]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:14<00:00,  1.03it/s, lowest_mse=0.00186, pred_loss=12.4, trigger=0]

LBFGS Biased
The train error is 12.36 for SOAP
The test error is 23.16 for SOAP


#### Bulk - 0.1 SOAP

In [413]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:15<00:00,  1.02s/it, lowest_mse=0.0166, pred_loss=32.8, trigger=0]

LBFGS Unbiased
The train error is 32.48 for SOAP
The test error is 38.2 for SOAP


In [414]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(bulk_soap, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:14<00:00,  1.03it/s, lowest_mse=0.0169, pred_loss=31.8, trigger=0]

LBFGS Biased
The train error is 31.54 for SOAP
The test error is 44.99 for SOAP


#### Bulk - 0.3 Kernel

In [415]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_200, bulk_dos3,
                                                            bulk_train_index, bulk_test_index,
                                                            bulk_kMM_200, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.69it/s, lowest_mse=0.00421, pred_loss=19.4, trigger=0]

LBFGS Unbiased
The train error is 19.42 for Kernel
The test error is 20.25 for Kernel


In [416]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_100, bulk_dos3,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            bulk_kMM_100, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.86it/s, lowest_mse=0.00529, pred_loss=20.9, trigger=0]

LBFGS Biased
The train error is 20.88 for Kernel
The test error is 28.45 for Kernel


#### Bulk - 0.1 Kernel

In [417]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_200, bulk_dos1,
                                                            bulk_train_index, bulk_test_index,
                                                            bulk_kMM_200, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.25it/s, lowest_mse=0.0245, pred_loss=39.9, trigger=0]

LBFGS Unbiased
The train error is 39.86 for Kernel
The test error is 41.38 for Kernel


In [418]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(bulk_kernel_100, bulk_dos1,
                                                            bulk_biased_train_index, bulk_biased_test_index,
                                                            bulk_kMM_100, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.87it/s, lowest_mse=0.0278, pred_loss=40.7, trigger=0]

LBFGS Biased
The train error is 40.71 for Kernel
The test error is 50.18 for Kernel


### Total

#### Total - 0.3 SOAP

In [419]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:13<00:00,  1.11it/s, lowest_mse=0.00532, pred_loss=19.5, trigger=0]

LBFGS Unbiased
The train error is 19.46 for SOAP
The test error is 23.15 for SOAP


In [420]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.20it/s, lowest_mse=0.00531, pred_loss=18.6, trigger=0]

LBFGS Biased
The train error is 18.33 for SOAP
The test error is 28.4 for SOAP


In [421]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.20it/s, lowest_mse=0.0043, pred_loss=17.7, trigger=0]

LBFGS Holdout
The train error is 17.68 for SOAP
The test error is 32.62 for SOAP


#### Total - 0.1 SOAP

In [422]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:13<00:00,  1.08it/s, lowest_mse=0.0208, pred_loss=33.7, trigger=0]

LBFGS Unbiased
The train error is 33.45 for SOAP
The test error is 37.42 for SOAP


In [423]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.20it/s, lowest_mse=0.0212, pred_loss=32.6, trigger=0]

LBFGS Biased
The train error is 32.42 for SOAP
The test error is 43.5 for SOAP


In [424]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.19it/s, lowest_mse=0.0199, pred_loss=33.1, trigger=0]

LBFGS Holdout
The train error is 32.88 for SOAP
The test error is 42.94 for SOAP


#### Total - 0.3 Kernel 

In [425]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos3,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.37it/s, lowest_mse=0.0101, pred_loss=26.8, trigger=0]

LBFGS Unbiased
The train error is 26.83 for Kernel
The test error is 29.08 for Kernel


In [426]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.38it/s, lowest_mse=0.011, pred_loss=26.8, trigger=0]

LBFGS Biased
The train error is 26.82 for Kernel
The test error is 35.53 for Kernel


In [427]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_100, total_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_100, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.61it/s, lowest_mse=0.00805, pred_loss=24.2, trigger=0]

LBFGS Holdout
The train error is 24.19 for Kernel
The test error is 37.8 for Kernel


#### Total - 0.1 Kernel

In [428]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:08<00:00,  1.77it/s, lowest_mse=0.0288, pred_loss=39.6, trigger=0]

LBFGS Unbiased
The train error is 39.06 for Kernel
The test error is 40.75 for Kernel


In [429]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  1.91it/s, lowest_mse=0.0306, pred_loss=39.2, trigger=0]

LBFGS Biased
The train error is 39.19 for Kernel
The test error is 48.14 for Kernel


In [430]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.41it/s, lowest_mse=0.0285, pred_loss=39.6, trigger=0]

LBFGS Holdout
The train error is 39.62 for Kernel
The test error is 48.15 for Kernel


### Total Aligned

#### Total Aligned - 0.3 SOAP

In [431]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.24it/s, lowest_mse=0.00467, pred_loss=18.6, trigger=0]

LBFGS Unbiased
The train error is 18.56 for SOAP
The test error is 22.18 for SOAP


In [432]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:11<00:00,  1.26it/s, lowest_mse=0.00464, pred_loss=17.7, trigger=0]

LBFGS Biased
The train error is 17.4 for SOAP
The test error is 27.59 for SOAP


In [433]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:12<00:00,  1.24it/s, lowest_mse=0.00408, pred_loss=17.5, trigger=0]

LBFGS Holdout
The train error is 17.22 for SOAP
The test error is 30.12 for SOAP


#### Total Aligned - 0.1 SOAP

In [434]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            1e-2, 45, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 44: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:41<00:00,  1.08it/s, lowest_mse=0.019, pred_loss=32.6, trigger=0]

LBFGS Unbiased
The train error is 32.62 for SOAP
The test error is 37.68 for SOAP


In [435]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:11<00:00,  1.27it/s, lowest_mse=0.0205, pred_loss=32.5, trigger=0]

LBFGS Biased
The train error is 32.39 for SOAP
The test error is 43.29 for SOAP


In [436]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(total_soap, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for SOAP".format(loss_dos))
print ("The test error is {:.4} for SOAP".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:11<00:00,  1.27it/s, lowest_mse=0.0196, pred_loss=33.2, trigger=0]

LBFGS Holdout
The train error is 33.07 for SOAP
The test error is 42.03 for SOAP


#### Total Aligned - 0.3 Kernel 

In [437]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos3,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.73it/s, lowest_mse=0.00954, pred_loss=26.5, trigger=0]

LBFGS Unbiased
The train error is 26.52 for Kernel
The test error is 28.03 for Kernel


In [438]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos3,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.35it/s, lowest_mse=0.00919, pred_loss=24.9, trigger=0]

LBFGS Biased
The train error is 24.91 for Kernel
The test error is 34.07 for Kernel


In [439]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_100, total_aligned_dos3,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_100, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.23it/s, lowest_mse=0.00773, pred_loss=24, trigger=0]

LBFGS Holdout
The train error is 24.02 for Kernel
The test error is 35.6 for Kernel


#### Total Aligned - 0.1 Kernel

In [440]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            total_train_index, total_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.73it/s, lowest_mse=0.029, pred_loss=40.3, trigger=0]

LBFGS Unbiased
The train error is 40.3 for Kernel
The test error is 42.13 for Kernel


In [441]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            total_biased_train_index, total_biased_test_index,
                                                            total_kMM_150, 1e-1, 15, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:07<00:00,  2.01it/s, lowest_mse=0.0302, pred_loss=39.5, trigger=0]

LBFGS Biased
The train error is 39.46 for Kernel
The test error is 48.59 for Kernel


In [442]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(total_kernel_150, total_aligned_dos1,
                                                            holdout_train_index, holdout_test_index,
                                                            total_kMM_150, 1e-2, 15, 1)
print ("LBFGS Holdout")
print ("The train error is {:.4} for Kernel".format(loss_dos))
print ("The test error is {:.4} for Kernel".format(test_loss_dos))

Epoch: 14: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.15it/s, lowest_mse=0.0271, pred_loss=39.1, trigger=0]

LBFGS Holdout
The train error is 39.07 for Kernel
The test error is 46.51 for Kernel


# Old stuff 

### 0.1eV Smearing

In [639]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos1, train_index, test_index, 1e-3, 10000, 16, 1e-2)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1007:  10%|██████████▎                                                                                           | 1007/10000 [03:51<29:40,  5.05it/s, lowest_mse=0.0467, pred_loss=27.5, trigger=0]

The batch_size is now:  32


Epoch: 1611:  16%|████████████████▍                                                                                     | 1611/10000 [05:05<13:38, 10.25it/s, lowest_mse=0.0466, pred_loss=27.4, trigger=0]

The batch_size is now:  64


Epoch: 2217:  22%|██████████████████████▌                                                                               | 2216/10000 [05:44<06:42, 19.33it/s, lowest_mse=0.0465, pred_loss=27.4, trigger=0]

...

The batch_size is now:  512


Epoch: 4032:  40%|█████████████████████████████████████████                                                             | 4025/10000 [06:37<01:54, 52.15it/s, lowest_mse=0.0463, pred_loss=27.3, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|███████████████████████████████████████████████▏                                                      | 4624/10000 [06:47<07:53, 11.34it/s, lowest_mse=0.0463, pred_loss=27.3, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 27.32 for SOAP
The test error is 26.9 for SOAP


In [640]:
A_U_weights_01, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos1, train_index, test_index, kMM, 1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 807:   8%|████████▌                                                                                                 | 807/10000 [01:51<15:39,  9.78it/s, lowest_mse=0.37, pred_loss=77.2, trigger=0]

The batch_size is now:  32


Epoch: 1413:  14%|██████████████▌                                                                                        | 1412/10000 [02:33<07:57, 17.99it/s, lowest_mse=0.328, pred_loss=72.8, trigger=0]

The batch_size is now:  64


Epoch: 2018:  20%|████████████████████▊                                                                                  | 2016/10000 [02:58<04:31, 29.43it/s, lowest_mse=0.302, pred_loss=69.8, trigger=0]

...

The batch_size is now:  512


Epoch: 3839:  38%|███████████████████████████████████████▌                                                               | 3836/10000 [03:32<01:12, 85.19it/s, lowest_mse=0.276, pred_loss=66.8, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▌                                                         | 4423/10000 [03:38<04:35, 20.21it/s, lowest_mse=0.274, pred_loss=66.4, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 66.45 for KERNEL
The test error is 66.88 for KERNEL


## Baseline GD model - Adam -biased

### 0.3eV Smearing

In [641]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos3, biased_train_index, biased_test_index, 1e-3, 10000, 16, 1e-2)
print ("Adam biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1007:  10%|██████████▏                                                                                          | 1007/10000 [03:39<32:13,  4.65it/s, lowest_mse=0.00615, pred_loss=12.9, trigger=0]

The batch_size is now:  32


Epoch: 1612:  16%|████████████████▎                                                                                    | 1611/10000 [04:51<12:49, 10.90it/s, lowest_mse=0.00613, pred_loss=12.8, trigger=0]

The batch_size is now:  64


Epoch: 2217:  22%|██████████████████████▌                                                                               | 2215/10000 [05:30<06:57, 18.64it/s, lowest_mse=0.0061, pred_loss=12.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4034:  40%|████████████████████████████████████████▋                                                            | 4028/10000 [06:23<01:43, 57.83it/s, lowest_mse=0.00605, pred_loss=12.8, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|██████████████████████████████████████████████▋                                                      | 4624/10000 [06:31<07:35, 11.81it/s, lowest_mse=0.00605, pred_loss=12.8, trigger=0]


The batch_size is now:  2048
Adam biased
The train error is 12.75 for SOAP
The test error is 16.67 for SOAP


In [642]:
A_U_weights_01, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos1, biased_train_index, biased_test_index, kMM, 1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 807:   8%|████████▍                                                                                                | 807/10000 [01:43<15:31,  9.87it/s, lowest_mse=0.381, pred_loss=76.1, trigger=0]

The batch_size is now:  32


Epoch: 1413:  14%|██████████████▌                                                                                        | 1412/10000 [02:25<07:53, 18.13it/s, lowest_mse=0.336, pred_loss=71.4, trigger=0]

The batch_size is now:  64


Epoch: 2018:  20%|████████████████████▊                                                                                  | 2016/10000 [02:49<04:28, 29.78it/s, lowest_mse=0.312, pred_loss=68.9, trigger=0]

...

The batch_size is now:  512


Epoch: 3839:  38%|████████████████████████████████████████▎                                                                | 3836/10000 [03:23<01:12, 85.06it/s, lowest_mse=0.287, pred_loss=66, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▌                                                         | 4423/10000 [03:29<04:24, 21.08it/s, lowest_mse=0.284, pred_loss=65.7, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 65.67 for KERNEL
The test error is 72.64 for KERNEL


### 0.1eV Smearing

In [643]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos1, biased_train_index, biased_test_index, 1e-3, 10000, 16, 1e-2)
print ("Adam biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1007:  10%|██████████▎                                                                                           | 1007/10000 [03:40<27:57,  5.36it/s, lowest_mse=0.0459, pred_loss=26.4, trigger=0]

The batch_size is now:  32


Epoch: 1612:  16%|████████████████▍                                                                                     | 1611/10000 [04:51<12:48, 10.92it/s, lowest_mse=0.0458, pred_loss=26.4, trigger=0]

The batch_size is now:  64


Epoch: 2217:  22%|██████████████████████▌                                                                               | 2216/10000 [05:29<06:40, 19.44it/s, lowest_mse=0.0457, pred_loss=26.3, trigger=0]

...

The batch_size is now:  512


Epoch: 4034:  40%|█████████████████████████████████████████                                                             | 4028/10000 [06:21<01:45, 56.55it/s, lowest_mse=0.0455, pred_loss=26.3, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|███████████████████████████████████████████████▏                                                      | 4624/10000 [06:29<07:32, 11.87it/s, lowest_mse=0.0455, pred_loss=26.3, trigger=0]

The batch_size is now:  2048
Adam biased
The train error is 26.27 for SOAP
The test error is 31.64 for SOAP


In [644]:
A_U_weights_01, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos1, biased_train_index, biased_test_index, kMM, 1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 807:   8%|████████▍                                                                                                | 807/10000 [01:44<15:46,  9.72it/s, lowest_mse=0.379, pred_loss=75.8, trigger=0]

The batch_size is now:  32


Epoch: 1413:  14%|██████████████▌                                                                                        | 1412/10000 [02:26<07:57, 17.98it/s, lowest_mse=0.337, pred_loss=71.5, trigger=0]

The batch_size is now:  64


Epoch: 2018:  20%|████████████████████▊                                                                                  | 2016/10000 [02:50<04:26, 29.94it/s, lowest_mse=0.312, pred_loss=68.7, trigger=0]

...

The batch_size is now:  512


Epoch: 3838:  38%|████████████████████████████████████████▎                                                                | 3837/10000 [03:27<01:15, 81.98it/s, lowest_mse=0.287, pred_loss=66, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▌                                                         | 4423/10000 [03:34<04:29, 20.66it/s, lowest_mse=0.284, pred_loss=65.7, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 65.67 for KERNEL
The test error is 72.58 for KERNEL


## Baseline GD model - LBFGS - unbiased

### 0.3eV Smearing

In [569]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos3, train_index, test_index, 1e-3, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  9.56it/s, lowest_mse=0.01, pred_loss=17, trigger=0]


LBFGS Unbiased
The train error is 17.04 for SOAP
The test error is 15.35 for SOAP


In [570]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos3, train_index, test_index, kMM, 1, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 18.77it/s, lowest_mse=0.0476, pred_loss=37.1, trigger=0]


LBFGS Unbiased
The train error is 37.09 for Kernel
The test error is 36.84 for Kernel


### 0.1eV Smearing

In [568]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos1, train_index, test_index, 1e-3, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.12it/s, lowest_mse=0.0438, pred_loss=26.6, trigger=0]


LBFGS Unbiased
The train error is 26.57 for SOAP
The test error is 26.29 for SOAP


In [571]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos1, train_index, test_index, kMM, 1, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 10.16it/s, lowest_mse=0.202, pred_loss=57, trigger=0]

LBFGS Unbiased
The train error is 57.02 for Kernel
The test error is 57.55 for Kernel


## Baseline GD model - LBFGS - biased

### 0.3eV Smearing

In [573]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos3, biased_train_index, biased_test_index, 1e-3, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 14.33it/s, lowest_mse=0.0111, pred_loss=17.3, trigger=0]

LBFGS Biased
The train error is 17.29 for SOAP
The test error is 22.66 for SOAP


In [578]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos3, biased_train_index, biased_test_index, kMM, 1, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 21.03it/s, lowest_mse=0.0486, pred_loss=36.2, trigger=0]

LBFGS Unbiased
The train error is 36.15 for Kernel
The test error is 41.55 for Kernel


### 0.1 eV Smearing

In [580]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos1, biased_train_index, biased_test_index, 1e-3, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 10.51it/s, lowest_mse=0.0586, pred_loss=29.8, trigger=0]


LBFGS Biased
The train error is 29.8 for SOAP
The test error is 35.63 for SOAP


In [579]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos1, biased_train_index, biased_test_index, kMM, 1, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 11.08it/s, lowest_mse=0.21, pred_loss=56.5, trigger=0]

LBFGS Biased
The train error is 56.48 for Kernel
The test error is 63.5 for Kernel


## Baseline GF model - Adam - Clusterless Unbiased

### 0.3eV Smearing

In [705]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos3, clusterless_train, clusterless_test, 1e-3, 10000, 16, 1e-2)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1008:  10%|██████████▏                                                                                          | 1008/10000 [02:45<19:29,  7.69it/s, lowest_mse=0.00551, pred_loss=14.1, trigger=0]

The batch_size is now:  32


Epoch: 1613:  16%|████████████████▌                                                                                      | 1613/10000 [03:39<09:47, 14.27it/s, lowest_mse=0.00549, pred_loss=14, trigger=0]

The batch_size is now:  64


Epoch: 2218:  22%|██████████████████████▊                                                                                | 2216/10000 [04:09<05:27, 23.75it/s, lowest_mse=0.00546, pred_loss=14, trigger=0]

...

The batch_size is now:  512


Epoch: 4037:  40%|████████████████████████████████████████▋                                                            | 4033/10000 [04:50<01:27, 68.52it/s, lowest_mse=0.00542, pred_loss=13.9, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|██████████████████████████████████████████████▋                                                      | 4624/10000 [04:57<05:46, 15.53it/s, lowest_mse=0.00541, pred_loss=13.9, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 13.88 for SOAP
The test error is 15.64 for SOAP


In [706]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos3, clusterless_train, clusterless_test, kMM, 1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 808:   8%|████████▍                                                                                                | 807/10000 [01:20<12:55, 11.85it/s, lowest_mse=0.123, pred_loss=66.5, trigger=0]

The batch_size is now:  32


Epoch: 1414:  14%|██████████████▌                                                                                        | 1414/10000 [01:53<06:09, 23.22it/s, lowest_mse=0.099, pred_loss=60.8, trigger=0]

The batch_size is now:  64


Epoch: 2021:  20%|████████████████████▌                                                                                 | 2021/10000 [02:12<03:16, 40.56it/s, lowest_mse=0.0796, pred_loss=53.2, trigger=0]

...

The batch_size is now:  512


Epoch: 3841:  38%|███████████████████████████████████████                                                               | 3834/10000 [02:40<01:06, 92.97it/s, lowest_mse=0.0627, pred_loss=47.2, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▉                                                          | 4423/10000 [02:45<03:28, 26.75it/s, lowest_mse=0.06, pred_loss=46.2, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 46.2 for KERNEL
The test error is 46.16 for KERNEL


### 0.1eV Smearing

In [707]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos1, clusterless_train, clusterless_test, 1e-3, 10000, 16, 1e-2)
print ("Adam Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1008:  10%|██████████▎                                                                                           | 1008/10000 [02:45<19:33,  7.66it/s, lowest_mse=0.0382, pred_loss=27.8, trigger=0]

The batch_size is now:  32


Epoch: 1613:  16%|████████████████▍                                                                                     | 1613/10000 [03:40<09:51, 14.18it/s, lowest_mse=0.0381, pred_loss=27.7, trigger=0]

The batch_size is now:  64


Epoch: 2219:  22%|██████████████████████▊                                                                                | 2218/10000 [04:10<05:04, 25.52it/s, lowest_mse=0.038, pred_loss=27.7, trigger=0]

...

The batch_size is now:  512


Epoch: 4037:  40%|█████████████████████████████████████████▏                                                            | 4033/10000 [04:54<01:27, 68.57it/s, lowest_mse=0.0379, pred_loss=27.6, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|███████████████████████████████████████████████▏                                                      | 4624/10000 [05:02<05:51, 15.28it/s, lowest_mse=0.0378, pred_loss=27.6, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 27.62 for SOAP
The test error is 32.43 for SOAP


In [708]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos1, clusterless_train, clusterless_test, kMM, 1, 10000, 16, 1e-3)

print ("Adam Unbiased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 808:   8%|████████▍                                                                                                | 808/10000 [01:23<12:13, 12.53it/s, lowest_mse=0.288, pred_loss=76.2, trigger=0]

The batch_size is now:  32


Epoch: 1414:  14%|██████████████▌                                                                                        | 1412/10000 [01:57<06:18, 22.70it/s, lowest_mse=0.251, pred_loss=71.2, trigger=0]

The batch_size is now:  64


Epoch: 2021:  20%|████████████████████▊                                                                                  | 2017/10000 [02:17<03:38, 36.51it/s, lowest_mse=0.227, pred_loss=67.7, trigger=0]

...

The batch_size is now:  512


Epoch: 3839:  38%|███████████████████████████████████████▌                                                               | 3838/10000 [02:48<01:09, 88.40it/s, lowest_mse=0.203, pred_loss=63.9, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▌                                                         | 4423/10000 [02:54<03:40, 25.32it/s, lowest_mse=0.199, pred_loss=63.3, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 63.34 for KERNEL
The test error is 64.01 for KERNEL


## Baseline GF model - Adam - Clusterless Biased

### 0.3eV Smearing

In [709]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos3, biased_clusterless_train_index, biased_clusterless_test_index, 1e-3, 10000, 16, 1e-2)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1008:  10%|██████████▏                                                                                          | 1008/10000 [02:47<19:44,  7.59it/s, lowest_mse=0.00535, pred_loss=17.9, trigger=0]

The batch_size is now:  32


Epoch: 1613:  16%|████████████████▎                                                                                    | 1613/10000 [03:41<09:49, 14.24it/s, lowest_mse=0.00533, pred_loss=17.8, trigger=0]

The batch_size is now:  64


Epoch: 2218:  22%|██████████████████████▍                                                                              | 2216/10000 [04:13<05:17, 24.52it/s, lowest_mse=0.00531, pred_loss=17.8, trigger=0]

...

The batch_size is now:  512


Epoch: 4037:  40%|████████████████████████████████████████▋                                                            | 4030/10000 [04:55<01:28, 67.38it/s, lowest_mse=0.00527, pred_loss=17.7, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|██████████████████████████████████████████████▋                                                      | 4624/10000 [05:03<05:52, 15.26it/s, lowest_mse=0.00526, pred_loss=17.7, trigger=0]

The batch_size is now:  2048
Adam Unbiased
The train error is 17.72 for SOAP
The test error is 23.06 for SOAP


In [710]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos3, biased_clusterless_train_index, biased_clusterless_test_index, kMM, 1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 808:   8%|████████▍                                                                                               | 808/10000 [01:21<12:18, 12.45it/s, lowest_mse=0.0661, pred_loss=62.8, trigger=0]

The batch_size is now:  32


Epoch: 1414:  14%|██████████████▍                                                                                       | 1412/10000 [01:55<06:26, 22.24it/s, lowest_mse=0.0549, pred_loss=57.2, trigger=0]

The batch_size is now:  64


Epoch: 2019:  20%|████████████████████▌                                                                                 | 2016/10000 [02:15<03:50, 34.66it/s, lowest_mse=0.0479, pred_loss=53.5, trigger=0]

...

The batch_size is now:  512


Epoch: 3842:  38%|███████████████████████████████████████                                                               | 3833/10000 [02:44<01:05, 94.72it/s, lowest_mse=0.0381, pred_loss=47.7, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████                                                         | 4423/10000 [02:49<03:34, 26.04it/s, lowest_mse=0.0363, pred_loss=46.5, trigger=0]


The batch_size is now:  2048
Adam Unbiased
The train error is 46.52 for KERNEL
The test error is 56.51 for KERNEL


### 0.1eV Smearing

In [711]:
weights, loss_dos, test_loss_dos = normal_reg_train_Ad(SOAP, ldos1, biased_clusterless_train_index, biased_clusterless_test_index, 1e-3, 10000, 16, 1e-2)
print ("Adam Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 1008:  10%|██████████▎                                                                                           | 1008/10000 [02:45<19:35,  7.65it/s, lowest_mse=0.0345, pred_loss=37.3, trigger=0]

The batch_size is now:  32


Epoch: 1612:  16%|████████████████▍                                                                                     | 1612/10000 [03:39<10:24, 13.43it/s, lowest_mse=0.0344, pred_loss=37.2, trigger=0]

The batch_size is now:  64


Epoch: 2219:  22%|██████████████████████▌                                                                               | 2216/10000 [04:10<05:25, 23.93it/s, lowest_mse=0.0344, pred_loss=37.2, trigger=0]

...

The batch_size is now:  512


Epoch: 4037:  40%|█████████████████████████████████████████▏                                                            | 4033/10000 [04:53<01:28, 67.55it/s, lowest_mse=0.0343, pred_loss=37.1, trigger=0]

The batch_size is now:  1024


Epoch: 4624:  46%|███████████████████████████████████████████████▏                                                      | 4624/10000 [05:01<05:50, 15.34it/s, lowest_mse=0.0343, pred_loss=37.1, trigger=0]


The batch_size is now:  2048
Adam Biased
The train error is 37.12 for SOAP
The test error is 42.79 for SOAP


In [712]:
A_U_weights_03, loss_dos, test_loss_dos = kernel_reg_train_Ad(KERNEL, ldos1, biased_clusterless_train_index, biased_clusterless_test_index, kMM, 1, 10000, 16, 1e-3)

print ("Adam Biased")
print ("The train error is {:.4} for KERNEL".format(loss_dos,m))
print ("The test error is {:.4} for KERNEL".format(test_loss_dos, m))

Epoch: 808:   8%|████████▍                                                                                                | 808/10000 [01:22<12:28, 12.28it/s, lowest_mse=0.128, pred_loss=72.1, trigger=0]

The batch_size is now:  32


Epoch: 1414:  14%|██████████████▌                                                                                        | 1414/10000 [01:59<05:54, 24.22it/s, lowest_mse=0.116, pred_loss=68.6, trigger=0]

The batch_size is now:  64


Epoch: 2021:  20%|████████████████████▊                                                                                  | 2019/10000 [02:18<03:26, 38.63it/s, lowest_mse=0.107, pred_loss=65.4, trigger=0]

...

The batch_size is now:  512


Epoch: 3841:  38%|███████████████████████████████████████▏                                                              | 3837/10000 [02:46<01:08, 90.40it/s, lowest_mse=0.0948, pred_loss=61.7, trigger=0]

The batch_size is now:  1024


Epoch: 4423:  44%|█████████████████████████████████████████████▉                                                          | 4423/10000 [02:52<03:37, 25.62it/s, lowest_mse=0.0926, pred_loss=61, trigger=0]

The batch_size is now:  2048
Adam Biased
The train error is 60.99 for KERNEL
The test error is 72.15 for KERNEL


## Baseline GF model - LBFGS - Clusterless Unbiased

### 0.3 eV Smearing

In [713]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos3, clusterless_train, clusterless_test, 1e-3, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 14.51it/s, lowest_mse=0.00774, pred_loss=16.6, trigger=0]

LBFGS Unbiased
The train error is 16.6 for SOAP
The test error is 17.22 for SOAP


In [725]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos3, clusterless_train, clusterless_test, kMM, 1, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 17.40it/s, lowest_mse=0.0342, pred_loss=34.9, trigger=0]

LBFGS Unbiased
The train error is 34.91 for Kernel
The test error is 35.23 for Kernel


### 0.1eV Smearing

In [726]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos1, clusterless_train, clusterless_test, 1e-3, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:04<00:00,  8.47it/s, lowest_mse=0.0338, pred_loss=26.1, trigger=0]

LBFGS Unbiased
The train error is 26.12 for SOAP
The test error is 31.62 for SOAP


In [727]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos1, clusterless_train, clusterless_test, kMM, 1, 40, 1)
print ("LBFGS Unbiased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 27.16it/s, lowest_mse=0.145, pred_loss=54, trigger=0]


LBFGS Unbiased
The train error is 54.02 for Kernel
The test error is 55.19 for Kernel


## Baseline GF model - LBFGS - Clusterless Biased

### 0.3eV Smearing

In [733]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos3, biased_clusterless_train_index, biased_clusterless_test_index, 1e-3, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 17.52it/s, lowest_mse=0.00751, pred_loss=21.2, trigger=0]


LBFGS Biased
The train error is 21.16 for SOAP
The test error is 26.97 for SOAP


In [729]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos3, biased_clusterless_train_index, biased_clusterless_test_index, kMM, 1, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 11.99it/s, lowest_mse=0.0193, pred_loss=33.9, trigger=0]

LBFGS Biased
The train error is 33.9 for Kernel
The test error is 41.52 for Kernel


### 0.1eV Smearing

In [730]:
U_L_weights3 , loss_dos, test_loss_dos = normal_reg_train_L(SOAP, ldos1, biased_clusterless_train_index, biased_clusterless_test_index, 1e-3, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for SOAP".format(loss_dos,m))
print ("The test error is {:.4} for SOAP".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 13.31it/s, lowest_mse=0.0355, pred_loss=37.8, trigger=0]

LBFGS Biased
The train error is 37.76 for SOAP
The test error is 45.28 for SOAP


In [731]:
U_L_weights3 , loss_dos, test_loss_dos = kernel_reg_train_L(KERNEL, ldos1, biased_clusterless_train_index, biased_clusterless_test_index, kMM, 1, 40, 1)
print ("LBFGS Biased")
print ("The train error is {:.4} for Kernel".format(loss_dos,m))
print ("The test error is {:.4} for Kernel".format(test_loss_dos, m))

Epoch: 39: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:03<00:00, 13.31it/s, lowest_mse=0.0638, pred_loss=50.7, trigger=0]

LBFGS Biased
The train error is 50.66 for Kernel
The test error is 59.31 for Kernel
